0. Importing Packages

In [ ]:
%matplotlib qt
import mne
import numpy as np
import pandas as pd
import os
import scipy
import seaborn as sns
import os.path as op
from math import pi
import shap
from itertools import product
from tqdm.notebook import tqdm
# from mne_icalabel import label_components
import mne_features
import matplotlib.pyplot as plt
import matplotlib
from fooof import FOOOFGroup
from fooof import FOOOF
import itertools
from scipy.stats import chi2_contingency
import fooof
from mne.datasets import fetch_fsaverage
from mne.minimum_norm import make_inverse_operator, apply_inverse, compute_source_psd_epochs, apply_inverse_epochs
from mne_connectivity import spectral_connectivity_time
from mne_connectivity.viz import plot_connectivity_circle
import joblib
import random

1. a. Loading and preprocessing the recordings (aaeeg data)

In [ ]:
# defining a function to find the nearest event
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

# selecting .vhdr files
directory = '/Users/payamsadeghishabestari/eeg_data/Regensburg/transfer_147155_files_ac07b771/aaeeg_rsEEG' 
files_list = []
for filename in sorted(os.listdir(directory)): ## iterate over files in that directory
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and f.endswith(".vhdr"):
        files_list.append(f)

# importing recordings all in a list
sfreq = 250
l_freq = .1 # Hz
h_freq = 80 # Hz
cut_out_sec = 3 # cutting out first few seconds to avoid eye related artifacts
epoch_duration = 2 # seconds

for file_vhdr in tqdm(files_list):
    raw_rs = mne.io.read_raw_brainvision(vhdr_fname=file_vhdr, preload=True, verbose=False)
    events = mne.events_from_annotations(raw_rs, verbose=False)
    
    # montaging
    new_ch_names = raw_rs.info['ch_names'].copy()
    new_ch_names[58] = 'Fpz'
    mapping = dict(zip(raw_rs.info['ch_names'], new_ch_names))
    mne.rename_channels(raw_rs.info, mapping)
    raw_rs.set_montage('standard_1020', verbose=False, on_missing='warn')
    
    # resampling and filtering
    raw_rs, events_resampled = raw_rs.resample(sfreq=sfreq, events=events[0], verbose=False)
    raw_rs = raw_rs.filter(l_freq=l_freq, h_freq=h_freq, verbose=False) 
    raw_rs = raw_rs.notch_filter(freqs=50, verbose=False) 
    
    # eye blink removal on whole recording
    ica_again = 0
    ica = mne.preprocessing.ICA(n_components=0.95, max_iter=800, method='infomax', fit_params=dict(extended=True))
    ica.fit(raw_rs, verbose=False) 
    ic_labels = label_components(raw_rs, ica, method="iclabel")
    labels = ic_labels["labels"]
    exclude_idxs = [idx for idx, label in enumerate(labels) if label in ["eye blink"]] 
    ica.apply(raw_rs, exclude=exclude_idxs, verbose=False)
    if len(exclude_idxs)==0:
        print(f'could not find blinking artifact in subject {file_vhdr[-14:-10]}')
        ica_again = 1
    
    # ec/eo stimuli timing
    ec_start = np.array([event[0] for event in events_resampled if event[2]==11])  # stimulus 11: eyes_close
    eo_start = np.array([event[0] for event in events_resampled if event[2]==31])  # stimulus 31: eyes_open

    # fixing for some subjects:
    if len(ec_start) == 4: # for subject index 26
        ec_start = np.insert(ec_start, 0, 0) 
        eo_start = np.insert(eo_start, 2, 78939)
    
    if len(eo_start) == 4: # for subject indeices 0, 2, 6, 7, 15, 16, 17, 20, 21, 23, 24, 25, 28, 30, 33, 34, 43
        possible_events = []
        if eo_start[0] > 150 * sfreq:
            for event in events_resampled:
                if event[2]==33:
                    if eo_start[0] > event[0]:
                        possible_events.append(event[0])
            possible_event = find_nearest(np.array(possible_events), 60 * sfreq)
            eo_start = np.insert(eo_start, 0, possible_event)
        
        if eo_start[-1] < 500 * sfreq:
            for event in events_resampled:
                if event[2]==33:
                    if eo_start[-1] < event[0]:
                        possible_events.append(event[0])
            possible_event = find_nearest(np.array(possible_events), 570 * sfreq)
            eo_start = np.append(eo_start, possible_event)
        
        if (eo_start[0] < 150 * sfreq) and (eo_start[-1] > 500 * sfreq) and (len(eo_start) == 4):
            index = np.where(np.diff(eo_start) > 200 * sfreq)[0][0] # where discontinuety happend
            for event in events_resampled:
                if event[2]==33:
                    if eo_start[index+1] > event[0] & event[0] > eo_start[index]:
                        possible_events.append(event[0])
            possible_event = find_nearest(np.array(possible_events), (eo_start[3] + eo_start[2]) / 2)
            eo_start = np.insert(eo_start, index + 1, possible_event)
    
        ec_start_t = ec_start / sfreq
        eo_start_t = eo_start / sfreq
        print(ec_start_t)
        print(eo_start_t)
    # cropping and concatenating the data
    cropped_raw_eo = []
    cropped_raw_ec = []
    if ec_start[0] <= eo_start[0]:
        for ec_st, eo_st in zip(ec_start_t, eo_start_t):
            cropped_raw_ec.append(raw_rs.copy().crop(tmin=ec_st + cut_out_sec, tmax=eo_st))
        for ec_st, eo_st in zip(ec_start_t[1:], eo_start_t[:-1]):
            cropped_raw_eo.append(raw_rs.copy().crop(tmin=eo_st + 3, tmax=ec_st))
    else:
        for ec_st, eo_st in zip(ec_start_t, eo_start_t):
            cropped_raw_eo.append(raw_rs.copy().crop(tmin=eo_st + 3, tmax=ec_st))
        for ec_st, eo_st in zip(ec_start_t[:-1], eo_start_t[1:]):
            cropped_raw_ec.append(raw_rs.copy().crop(tmin=ec_st + 3, tmax=eo_st))

    raw_ec_concat = cropped_raw_ec[0]
    raw_eo_concat = cropped_raw_eo[0]
    for raw_cr in cropped_raw_ec[1:]:
        raw_ec_concat = mne.io.concatenate_raws([raw_ec_concat, raw_cr], on_mismatch='ignore')
    for raw_cr in cropped_raw_eo[1:]:
        raw_eo_concat = mne.io.concatenate_raws([raw_eo_concat, raw_cr], on_mismatch='ignore')

    # applying ica again on only eo condition, if no blink found previously
    if ica_again == 1:
        ica = mne.preprocessing.ICA(n_components=0.95, max_iter=800, method='infomax', fit_params=dict(extended=True))
        ica.fit(raw_eo_concat, verbose=False) 
        ic_labels = label_components(raw_eo_concat, ica, method="iclabel")
        labels = ic_labels["labels"]
        exclude_idxs = [idx for idx, label in enumerate(labels) if label in ["eye blink"]] 
        ica.apply(raw_eo_concat, exclude=exclude_idxs, verbose=False)
    
    # average re-referencing
    raw_eo_concat.set_eeg_reference(ref_channels='average', projection=False, verbose=False)
    raw_ec_concat.set_eeg_reference(ref_channels='average', projection=False, verbose=False)
    
    # epoching (2- concat and epoching into 2 seconds)
    epochs_eo = mne.make_fixed_length_epochs(raw_eo_concat, duration=epoch_duration, preload=True)
    epochs_ec = mne.make_fixed_length_epochs(raw_ec_concat, duration=epoch_duration, preload=True)
    
    # saving epochs
    fname_save_eo = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/transfer_147155_files_ac07b771/aaeeg_rsEEG/epochs/{file_vhdr[-14:-10]}-eo-epo.fif' 
    fname_save_ec = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/transfer_147155_files_ac07b771/aaeeg_rsEEG/epochs/{file_vhdr[-14:-10]}-ec-epo.fif' 
    epochs_eo.save(fname=fname_save_eo, overwrite=True, verbose=False)
    epochs_ec.save(fname=fname_save_ec, overwrite=True, verbose=False)

1. b. Loading and preprocessing the recordings (neuropren data)

In [ ]:
# selecting .vhdr files
directory = '/Users/payamsadeghishabestari/eeg_data/Regensburg/transfer_147155_files_ac07b771/neuropren_rsEEG' 
files_list = []
for filename in sorted(os.listdir(directory)): ## iterate over files in that directory
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and f.endswith(".vhdr"):
        files_list.append(f)

# importing recordings all in a list
sfreq = 250
l_freq = .1 # Hz
h_freq = 80 # Hz
cut_out_sec = 3 # cutting out first few seconds to avoid eye related artifacts
epoch_duration = 2 # seconds

for file_vhdr in tqdm(files_list[17:]):
    print(file_vhdr)
    raw_rs = mne.io.read_raw_brainvision(vhdr_fname=file_vhdr, preload=True, verbose=False)
    events = mne.events_from_annotations(raw_rs, verbose=False)
    
    # montaging (removing ecg channels for now)
    new_ch_names = raw_rs.info['ch_names'].copy()
    new_ch_names[58] = 'Fpz' # change FPz to Fpz
    mapping = dict(zip(raw_rs.info['ch_names'], new_ch_names))
    mne.rename_channels(raw_rs.info, mapping)
    raw_rs.info["bads"].append("HRli")
    raw_rs.info["bads"].append("HRre")
    raw_rs.set_montage('standard_1020', verbose=False, on_missing='warn')
    
    # resampling and filtering
    raw_rs, events_resampled = raw_rs.resample(sfreq=sfreq, events=events[0], verbose=False)
    raw_rs = raw_rs.filter(l_freq=l_freq, h_freq=h_freq, verbose=False) 
    raw_rs = raw_rs.notch_filter(freqs=50, verbose=False) 
    
    # eye blink removal on whole recording
    ica_again = 0
    ica = mne.preprocessing.ICA(n_components=0.95, max_iter=800, method='infomax', fit_params=dict(extended=True))
    ica.fit(raw_rs, verbose=False) 
    ic_labels = label_components(raw_rs, ica, method="iclabel")
    labels = ic_labels["labels"]
    exclude_idxs = [idx for idx, label in enumerate(labels) if label in ["eye blink"]]
    ica.apply(raw_rs, exclude=exclude_idxs, verbose=False)
    if len(exclude_idxs)==0:
        print(f'could not find blinking artifact in subject {file_vhdr[-14:-10]}')
        ica_again = 1

    # ec/eo stimuli timing
    ec_start = np.array([event[0] for event in events_resampled if event[2]==44])  # stimulus 44: eyes_close
    eo_start = np.array([event[0] for event in events_resampled if event[2]==31])  # stimulus 31: eyes_open
    ec_init_start = np.array([event[0] for event in events_resampled if event[2]==10001]) # init stimulus for eyes close
    ec_start = np.insert(ec_start, 0, ec_init_start[0])

    # fixing for some subjects: (consider that subject indexes 13 and 19 have incorrect .mrk and .vhdr files)
    check = 0
    if (len(eo_start)==4) and (eo_start[-1] < 500 * sfreq): # last stim missing
        last_eo_stim = np.array([event[0] for event in events_resampled if event[2]==22])[-1]
        eo_start = np.append(eo_start, last_eo_stim)
        check = 1
    if (len(eo_start)==4) and (eo_start[0] > 100 * sfreq): # first stim missing
        first_eo_stim = np.array([event[0] for event in events_resampled if event[2]==22])[0]
        eo_start = np.insert(eo_start, 0, first_eo_stim)
        check = 1
    if (len(eo_start)==4) and check == 0:
        eo_start = np.array([event[0] for event in events_resampled if event[2]==22])

    ec_start_t = ec_start / sfreq
    eo_start_t = eo_start / sfreq
    
    # cropping and concatenating the data
    cropped_raw_eo = []
    cropped_raw_ec = []
    
    for ec_st, eo_st in zip(ec_start_t[:-1], eo_start_t):
        cropped_raw_ec.append(raw_rs.copy().crop(tmin=ec_st + cut_out_sec, tmax=eo_st))
    for ec_st, eo_st in zip(ec_start_t[1:], eo_start_t):
        cropped_raw_eo.append(raw_rs.copy().crop(tmin=eo_st + 3, tmax=ec_st))
    
    raw_ec_concat = cropped_raw_ec[0]
    raw_eo_concat = cropped_raw_eo[0]
    for raw_cr in cropped_raw_ec[1:]:
        raw_ec_concat = mne.io.concatenate_raws([raw_ec_concat, raw_cr], on_mismatch='ignore')
    for raw_cr in cropped_raw_eo[1:]:
        raw_eo_concat = mne.io.concatenate_raws([raw_eo_concat, raw_cr], on_mismatch='ignore')

    # applying ica again on only eo condition, if no blink found previously
    if ica_again == 1:
        ica = mne.preprocessing.ICA(n_components=0.95, max_iter=800, method='infomax', fit_params=dict(extended=True))
        ica.fit(raw_eo_concat, verbose=False) 
        ic_labels = label_components(raw_eo_concat, ica, method="iclabel")
        labels = ic_labels["labels"]
        exclude_idxs = [idx for idx, label in enumerate(labels) if label in ["eye blink"]] 
        ica.apply(raw_eo_concat, exclude=exclude_idxs, verbose=False)
    
    # average re-referencing
    raw_eo_concat.set_eeg_reference(ref_channels='average', projection=False, verbose=False)
    raw_ec_concat.set_eeg_reference(ref_channels='average', projection=False, verbose=False)
    
    # epoching (2- concat and epoching into 2 seconds)
    epochs_eo = mne.make_fixed_length_epochs(raw_eo_concat, duration=epoch_duration, preload=True)
    epochs_ec = mne.make_fixed_length_epochs(raw_ec_concat, duration=epoch_duration, preload=True)
    
    # saving epochs
    fname_save_eo = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/transfer_147155_files_ac07b771/neuropren_rsEEG/epochs/{file_vhdr[-17:-13]}-eo-epo.fif' 
    fname_save_ec = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/transfer_147155_files_ac07b771/neuropren_rsEEG/epochs/{file_vhdr[-17:-13]}-ec-epo.fif' 
    epochs_eo.save(fname=fname_save_eo, overwrite=True, verbose=False)
    epochs_ec.save(fname=fname_save_ec, overwrite=True, verbose=False)

1. c. Loading and preprocessing TEAS resting-state (preprocessed before)

In [ ]:
# selecting .fif files (epochs) (64 channels)
directory = '/Users/payamsadeghishabestari/eeg_data/teas/teas_rs_epochs' 
files_list = []
for filename in sorted(os.listdir(directory)): ## iterate over files in that directory
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and (f.endswith("_visit_3_run_RS1-ec-epo.fif") or f.endswith("_visit_3_run_rs1-ec-epo.fif")):
        files_list.append(f)
        
# subject 650 and 999 had no behavioral result 
epochs_ec = []
epochs_ec_labels = []
df_teas = pd.read_excel('/Users/payamsadeghishabestari/Prime/expertEAS_behav.xlsx')
df_v_0 = df_teas[df_teas['Visit']==0] # only look at visit 0
for file in files_list:
    epoch = mne.read_epochs(fname=file, preload=True, verbose=False)
    
    # montaging
    if epoch.info['ch_names'][-1] == 'Iz':
        old_channel_names = epoch.info['ch_names']
        new_channel_names = old_channel_names[:-1]  ### rename the Iz to FCz in our recordings
        new_channel_names.append('FCz')
        mapping = dict(zip(old_channel_names, new_channel_names))
        mne.rename_channels(epoch.info, mapping)
        epoch.set_montage('standard_1020', on_missing='raise')
    else:
        epoch.add_reference_channels(ref_channels=['FCz'])
        epoch.set_montage('standard_1020', on_missing='raise')
        epoch.info['bads'] = ['FCz']
        epoch.interpolate_bads(verbose=False)
    epochs_ec.append(epoch)
    
    # labeling
    sub_id = int(file[-30:-27])
    if len(df_v_0[df_v_0['patient']==sub_id]['RI'].values):
        epochs_ec_labels.append(len(epoch) * [df_v_0[df_v_0['patient']==sub_id]['RI'].values[0]])

# coding suppresion levels
for idx, i in enumerate(epochs_ec_labels):
    if all(j <= -1 for j in i):
        epochs_ec_labels[idx] = len(i) * [1]
        print('can suppress')
    if all(j > -1 for j in i):
        epochs_ec_labels[idx] = len(i) * [0]
        print('can not suppress')

# getting epochs data (each epoch shape is n_epochs * n_chs * n_times for each subject)
epochs_ec_data = []
for idx in range(len(epochs_ec)):
    epochs_ec_data.append(epochs_ec[idx].get_data(picks='eeg')) # not ecg channels

# cleaning and renaming for ML (choose ec or eo) ... for now I choose ec
data_list = epochs_ec_data
label_list = epochs_ec_labels
group_list = [[i]*len(j) for i,j in enumerate(data_list)]
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)
group_array = np.hstack(group_list)

In [ ]:
data_array.shape

1. d. Loading and preprocessing Tinnoice resting-state

In [ ]:
directory = '/Users/payamsadeghishabestari/eeg_data/tinnoice' 
files_list = []
for filename in sorted(os.listdir(directory)): ## iterate over files in that directory
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and f.endswith("run_1-ec_excl-ip-epo.fif"):
        files_list.append(f)

epochs_ec = []
epochs_ec_labels = []
df_tinnoice = pd.read_excel('/Users/payamsadeghishabestari/Prime/behav_tinnoice.xlsx')
for file in files_list:
    epoch = mne.read_epochs(file, preload=True, verbose=False)
    epoch.drop_channels(['1-EXG1', '1-EXG2', '1-EXG3', '1-EXG4',
                        '1-EXG5', '1-EXG6', '1-EXG7', '1-EXG8', '1-GSR1',
                        '1-GSR2', '1-Erg1', '1-Erg2', '1-Resp', '1-Plet', '1-Temp'], on_missing='warn')
    epoch.set_montage("biosemi128", verbose=False)
    epochs_ec.append(epoch)
    
    # labeling
    sub_id = int(file[56:59])
    if len(df_tinnoice[df_tinnoice['ID']==sub_id]['RI'].values):
        epochs_ec_labels.append(len(epoch) * [df_tinnoice[df_tinnoice['ID']==sub_id]['RI'].values[0]])

# coding suppresion levels
for idx, i in enumerate(epochs_ec_labels):
    if all(j <= -1 for j in i):
        epochs_ec_labels[idx] = len(i) * [1]
        print('can suppress')
    if all(j > -1 for j in i):
        epochs_ec_labels[idx] = len(i) * [0]
        print('can not suppress')

# getting epochs data (each epoch shape is n_epochs * n_chs * n_times for each subject)
epochs_ec_data = []
for idx in range(len(epochs_ec)):
    epochs_ec_data.append(epochs_ec[idx].get_data(picks='eeg')) # not ecg channels

# cleaning and renaming for ML (choose ec or eo) ... for now I choose ec
data_list = epochs_ec_data
label_list = epochs_ec_labels
group_list = [[i]*len(j) for i,j in enumerate(data_list)]
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)
group_array = np.hstack(group_list)

In [ ]:
data_array.shape

2. a. Loading epochs, combining two lists and labeling (check thr=90 for inhibition)

In [ ]:
# loading epochs and excel files (subject 'fvlz' is missing from excel; I changed subject 'fvlZ' in excel file to 'fvlz')
directory_aaeeg = '/Users/payamsadeghishabestari/eeg_data/Regensburg/transfer_147155_files_ac07b771/aaeeg_rsEEG/epochs' 
directory_neuropren = '/Users/payamsadeghishabestari/eeg_data/Regensburg/transfer_147155_files_ac07b771/neuropren_rsEEG/epochs' 
df_aaeeg = pd.read_excel('/Users/payamsadeghishabestari/eeg_data/Regensburg/transfer_147155_files_ac07b771/aaeeg_rsEEG/AAEEG_TL_data.xlsx')
df_neuropren = pd.read_excel('/Users/payamsadeghishabestari/eeg_data/Regensburg/transfer_147155_files_ac07b771/neuropren_rsEEG/neuropren_TL_data.xlsx')
files_list_eo = []; files_list_ec = []
epochs_eo = []; epochs_ec = []
epochs_eo_labels = []; epochs_ec_labels = []

# combining two datasets (aaeeg and neuropren)
for filename in sorted(os.listdir(directory_aaeeg)): ## iterate over files in that directory
    f = os.path.join(directory_aaeeg, filename)
    # checking if it is a file
    if (os.path.isfile(f)) and (f.endswith(".fif")) and (f[-10:-8] == 'eo'):
        files_list_eo.append(f)
    if (os.path.isfile(f)) and (f.endswith(".fif")) and (f[-10:-8] == 'ec'):
        files_list_ec.append(f)
for filename in sorted(os.listdir(directory_neuropren)): ## iterate over files in that directory
    f = os.path.join(directory_neuropren, filename)
    # checking if it is a file
    if (os.path.isfile(f)) and (f.endswith(".fif")) and (f[-10:-8] == 'eo'):
        files_list_eo.append(f)
    if (os.path.isfile(f)) and (f.endswith(".fif")) and (f[-10:-8] == 'ec'):
        files_list_ec.append(f)

# labeling
for file in files_list_eo:
    epoch_eo = mne.read_epochs(fname=file, preload=True, verbose=False)
    epoch_eo.drop_channels(['HRli', 'HRre'], on_missing='warn')
    epochs_eo.append(epoch_eo)
    
    if len(df_aaeeg[df_aaeeg['random_id']==file[-15:-11]]['WN_TL_0'].values):
        epochs_eo_labels.append(len(epoch_eo) * [df_aaeeg[df_aaeeg['random_id']==file[-15:-11]]['WN_TL_0'].values[0]])
    if len(df_neuropren[df_neuropren['random_id']==file[-15:-11]]['b1_WN_0'].values):
        epochs_eo_labels.append(len(epoch_eo) * [df_neuropren[df_neuropren['random_id']==file[-15:-11]]['b1_WN_0'].values[0]])   

for file in files_list_ec:
    epoch_ec = mne.read_epochs(fname=file, preload=True, verbose=False)
    epoch_ec.drop_channels(['HRli', 'HRre'], on_missing='warn')
    epochs_ec.append(epoch_ec)
    
    if len(df_aaeeg[df_aaeeg['random_id']==file[-15:-11]]['WN_TL_0'].values):
        epochs_ec_labels.append(len(epoch_ec) * [df_aaeeg[df_aaeeg['random_id']==file[-15:-11]]['WN_TL_0'].values[0]])
    if len(df_neuropren[df_neuropren['random_id']==file[-15: -11]]['b1_WN_0'].values):
        epochs_ec_labels.append(len(epoch_ec) * [df_neuropren[df_neuropren['random_id']==file[-15:-11]]['b1_WN_0'].values[0]]) 

# coding suppresion levels
for idx, i in enumerate(epochs_eo_labels):
    if all(j > 90 for j in i):
        epochs_eo_labels[idx] = len(i) * [0]
    if all(j <= 90 for j in i):
        epochs_eo_labels[idx] = len(i) * [1]

for idx, i in enumerate(epochs_ec_labels):
    if all(j > 90 for j in i):
        epochs_ec_labels[idx] = len(i) * [0]
    if all(j <= 90 for j in i):
        epochs_ec_labels[idx] = len(i) * [1]


####### multi class
#for idx, i in enumerate(epochs_eo_labels):
#    if all(90 < j for j in i):
#        epochs_eo_labels[idx] = len(i) * [0]
#    if all(70 < j <= 90 for j in i):
#        epochs_eo_labels[idx] = len(i) * [1]
#    if all(50 < j <= 70 for j in i):
#        epochs_eo_labels[idx] = len(i) * [2]
#    if all(j <= 50 for j in i):
#        epochs_eo_labels[idx] = len(i) * [3]

#for idx, i in enumerate(epochs_ec_labels):
#    if all(90 < j for j in i):
#        epochs_ec_labels[idx] = len(i) * [0]
#    if all(70 < j <= 90 for j in i):
#        epochs_ec_labels[idx] = len(i) * [1]
#    if all(50 < j <= 70 for j in i):
#        epochs_ec_labels[idx] = len(i) * [2]
#    if all(j <= 50 for j in i):
#        epochs_ec_labels[idx] = len(i) * [3]


# getting epochs data (each epoch shape is n_epochs * n_chs * n_times for each subject)
epochs_ec_data = []
epochs_eo_data = []
for idx in range(len(epochs_ec)):
    epochs_ec_data.append(epochs_ec[idx].get_data(picks='eeg')) # not ecg channels
    epochs_eo_data.append(epochs_eo[idx].get_data(picks='eeg')) # not ecg channels

# cleaning and renaming for ML (choose ec or eo) ... for now I choose ec
data_list = epochs_ec_data
label_list = epochs_ec_labels
group_list = [[i]*len(j) for i,j in enumerate(data_list)]
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)
group_array = np.hstack(group_list)

2. b. Checking data array size from combining lists: should be n_epochs * n_channels * n_times

In [ ]:
data_array.shape

3. a. Feature extraction (PSD in channels = 62*5 = 310)

In [ ]:
# function for extracting power bands in each channel
def eeg_power_band(epochs):
    """
    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn. the output of function is a numpy array of shape (n_samples, 5)
    """
    FREQ_BANDS = {"delta": [0.5, 4.5], "theta": [4.5, 8.5], "alpha": [8.5, 13.5],
                "beta": [15.5, 30], "gamma": [30, 80]}
    
    psds, freqs = epochs.compute_psd(method='multitaper', picks='eeg', fmin=0.5, fmax=80, verbose=False).get_data(return_freqs=True)
    psds /= np.sum(psds, axis=-1, keepdims=True)  # normalize the PSDs
    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band)
    print(len(psds_band[0]))
    print(len(X))
    print(len(X[0][0]))
    return np.concatenate(X, axis=1)

3. b. Feature extraction (Power in labels using minimu norm dSPM = 68*5 = 340)

In [ ]:
# making equal channel size
for ep in epochs_ec:
    ep.drop_channels(['HRli', 'HRre'], on_missing='warn')

# Compute PSD in brain labels of inverse solution from single epochs
fs_dir = fetch_fsaverage(verbose=False)
trans = "fsaverage"  
src = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
bem = op.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 69 labels
# brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc.a2009s')[:-2]
method = "dSPM"
snr = 3.0
lambda2 = 1.0 / snr**2
counter = 0
for ep in tqdm(epochs_ec[:30]):
    ep.set_eeg_reference('average', projection=True)
    fwd = mne.make_forward_solution(ep.info, trans=trans, src=src, bem=bem, eeg=True, mindist=5.0, verbose=False)
    # noise_cov = mne.compute_covariance(ep, tmax=0.0, method=["shrunk", "empirical"], rank=None, verbose=False)
    noise_cov = mne.make_ad_hoc_cov(ep.info, std=None, verbose=False)
    inverse_operator = make_inverse_operator(ep.info, fwd, noise_cov, loose=0.2, depth=0.8, verbose=False)
    label_ts = []
    for bl in tqdm(brain_labels):
        stc_power = compute_source_psd_epochs(ep, inverse_operator, lambda2=lambda2,
                                                    method=method, fmin=0.5, fmax=80, label=bl,
                                                    return_generator=False, verbose=False)
        label_ts.append(mne.extract_label_time_course(stc_power, bl, inverse_operator['src'], mode='mean',
                                                allow_empty=True, return_generator=False, verbose=False))
    
    file = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/labels_ts_teas_mean/label_ts_{counter}'
    np.save(file=file, arr=np.array(label_ts))      
    counter += 1

In [ ]:
# loading and extracting power bands in each  brain label
all_deltas = []; all_thetas = []; all_alphas = []; all_betas = []; all_gammas = []
for counter_idx in tqdm(range(29)):
    # if counter_idx != 15: # just for removing subject 555 in teas   
        file = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/labels_ts_teas_mean/label_ts_{counter_idx}.npy'
        arr = np.squeeze(np.load(file=file, allow_pickle=True))
        deltas = []; thetas = []; alphas = []; betas = []; gammas = []
        for ch_idx in range(arr.shape[0]):
            delta = []; theta = []; alpha = []; beta = []; gamma = []
            for epo_idx in range(arr.shape[1]):
                delta.append(np.average(arr[ch_idx][epo_idx][0:8]))
                theta.append(np.average(arr[ch_idx][epo_idx][8:16]))
                alpha.append(np.average(arr[ch_idx][epo_idx][16:26]))
                beta.append(np.average(arr[ch_idx][epo_idx][30:59]))
                gamma.append(np.average(arr[ch_idx][epo_idx][59:159]))
            
            deltas.append(delta)
            thetas.append(theta)
            alphas.append(alpha)
            betas.append(beta)
            gammas.append(gamma)
        
        all_deltas.append(deltas)
        all_thetas.append(thetas)
        all_alphas.append(alphas)
        all_betas.append(betas)
        all_gammas.append(gammas)

delta_powers_array = np.concatenate(all_deltas, axis=1).T
theta_powers_array = np.concatenate(all_thetas, axis=1).T
alpha_powers_array = np.concatenate(all_alphas, axis=1).T
beta_powers_array = np.concatenate(all_betas, axis=1).T
gamma_powers_array = np.concatenate(all_gammas, axis=1).T
power_brain_features = np.concatenate([delta_powers_array, theta_powers_array,
                alpha_powers_array, beta_powers_array, gamma_powers_array], axis=1)

3. b. 2. splitting to more band powers

In [ ]:
# loading and extracting power bands in each  brain label
all_deltas = []; all_thetas = []; all_l_alphas = []; all_u_alphas = []
all_m_betas = []; all_u_betas = []; all_l_gammas = []; all_u_gammas = []

for counter_idx in tqdm(range(73)):
    file = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/labels_ts/label_ts_{counter_idx}.npy'
    arr = np.squeeze(np.load(file=file, allow_pickle=True))
    deltas = []; thetas = []; l_alphas = []; u_alphas = []
    m_betas = []; u_betas = []; l_gammas = []; u_gammas = [] 
    for ch_idx in range(arr.shape[0]):
        delta = []; theta = []; l_alpha = []; u_alpha = []
        m_beta = []; u_beta = []; l_gamma = []; u_gamma = []
        for epo_idx in range(arr.shape[1]):
            delta.append(np.average(arr[ch_idx][epo_idx][0:8])) # 0.5 - 4.5 Hz
            theta.append(np.average(arr[ch_idx][epo_idx][8:16])) # 4.5 - 8.5 Hz
            l_alpha.append(np.average(arr[ch_idx][epo_idx][16:20])) # 8.5 - 10.5 Hz
            u_alpha.append(np.average(arr[ch_idx][epo_idx][20:26])) # 10.5 - 13.5 Hz
            m_beta.append(np.average(arr[ch_idx][epo_idx][29:39])) # 15 - 20 Hz 
            u_beta.append(np.average(arr[ch_idx][epo_idx][39:59])) # 20 - 30 Hz
            l_gamma.append(np.average(arr[ch_idx][epo_idx][59:99])) # 30 - 50 Hz
            u_gamma.append(np.average(arr[ch_idx][epo_idx][99:159])) # 50 - 80 Hz 
        
        deltas.append(delta)
        thetas.append(theta)
        l_alphas.append(l_alpha)
        u_alphas.append(u_alpha)
        m_betas.append(m_beta)
        u_betas.append(u_beta)
        l_gammas.append(l_gamma)
        u_gammas.append(u_gamma)
    
    all_deltas.append(deltas)
    all_thetas.append(thetas)
    all_l_alphas.append(l_alphas)
    all_u_alphas.append(u_alphas)
    all_m_betas.append(m_betas)
    all_u_betas.append(u_betas)
    all_l_gammas.append(l_gammas)
    all_u_gammas.append(u_gammas)

delta_powers_array = np.concatenate(all_deltas, axis=1).T
theta_powers_array = np.concatenate(all_thetas, axis=1).T
alpha_l_powers_array = np.concatenate(all_l_alphas, axis=1).T
alpha_u_powers_array = np.concatenate(all_u_alphas, axis=1).T
beta_m_powers_array = np.concatenate(all_m_betas, axis=1).T
beta_u_powers_array = np.concatenate(all_u_betas, axis=1).T
gamma_l_powers_array = np.concatenate(all_l_gammas, axis=1).T
gamma_u_powers_array = np.concatenate(all_u_gammas, axis=1).T

power_brain_freq_split_features = np.concatenate([delta_powers_array, theta_powers_array,
                                                alpha_l_powers_array, alpha_u_powers_array,
                                                beta_m_powers_array, beta_u_powers_array,
                                                gamma_l_powers_array, gamma_u_powers_array], axis=1)

3. c. Feature extraction (Shannon entropy = 62 * 5 = 310)

In [ ]:
# making equal channel size
for ep in epochs_ec:
    ep.drop_channels(['HRli', 'HRre'], on_missing='warn')

# computing app entropy features
print('computing entropy features (be patient) ...')
features_ent = []
for ep in tqdm(range(len(epochs_ec))): # get features from each epoch and save in a list
    for idx in range(len(epochs_ec[ep])):
        features_ent.append(mne_features.univariate.compute_app_entropy(epochs_ec[ep][idx].get_data()[0], emb=2, metric='chebyshev'))
features_ent_array = np.array(features_ent) # shape of (10332, 62) (nep * n_chs)
fname = '/Users/payamsadeghishabestari/eeg_data/Regensburg/features_ent_array.npy'
np.save(file=fname, arr=features_ent_array)

In [ ]:
# computing shannon entropy for each frequency band
# making equal channel size
for ep in epochs_ec:
    ep.drop_channels(['HRli', 'HRre'], on_missing='warn')

features_ent = []
deltas_ent = []; thetas_ent = []; alphas_ent = []; betas_ent = []; gammas_ent = [] 
for ep in tqdm(range(len(epochs_ec))): # get features from each epoch and save in a list
    delta_ent = []; theta_ent = []; alpha_ent = []; beta_ent = []; gamma_ent = [] 
    epochs_ec_delta = epochs_ec[ep].copy().filter(0.5, 4.5,verbose=False)
    
    for idx in range(len(epochs_ec_delta)):
        delta_ent.append(mne_features.univariate.compute_spect_entropy(sfreq=250, data=epochs_ec_delta[idx].get_data()[0], psd_method='multitaper'))

    epochs_ec_theta = epochs_ec[ep].copy().filter(4.5, 8.5,verbose=False)
    for idx in range(len(epochs_ec_theta)):
        theta_ent.append(mne_features.univariate.compute_spect_entropy(sfreq=250, data=epochs_ec_theta[idx].get_data()[0], psd_method='multitaper'))

    epochs_ec_alpha = epochs_ec[ep].copy().filter(8.5, 13.5,verbose=False)
    for idx in range(len(epochs_ec_alpha)):
        alpha_ent.append(mne_features.univariate.compute_spect_entropy(sfreq=250, data=epochs_ec_alpha[idx].get_data()[0], psd_method='multitaper'))

    epochs_ec_beta = epochs_ec[ep].copy().filter(15, 30,verbose=False)
    for idx in range(len(epochs_ec_beta)):
        beta_ent.append(mne_features.univariate.compute_spect_entropy(sfreq=250, data=epochs_ec_beta[idx].get_data()[0], psd_method='multitaper'))

    epochs_ec_gamma = epochs_ec[ep].copy().filter(30, 80,verbose=False)
    for idx in range(len(epochs_ec_gamma)):
        gamma_ent.append(mne_features.univariate.compute_spect_entropy(sfreq=250, data=epochs_ec_gamma[idx].get_data()[0], psd_method='multitaper'))

    deltas_ent.append(delta_ent)
    thetas_ent.append(theta_ent)
    alphas_ent.append(alpha_ent)
    betas_ent.append(beta_ent)
    gammas_ent.append(gamma_ent)

delta_feature = np.concatenate(deltas_ent, axis=0)
theta_feature = np.concatenate(thetas_ent, axis=0)
alpha_feature = np.concatenate(alphas_ent, axis=0)
beta_feature = np.concatenate(betas_ent, axis=0)
gamma_feature = np.concatenate(gammas_ent, axis=0)
entropy_features_array = np.concatenate((delta_feature, theta_feature, alpha_feature, beta_feature, gamma_feature), axis=1)
entropy_features_array.shape

In [ ]:
np.save('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_tinnoice_ent.npy', entropy_features_array)

3. d. Feature extraction (fooof aperiodic = 62 * 2 = 124)

In [ ]:
# making equal channel size
for ep in epochs_ec:
    ep.drop_channels(['HRli', 'HRre'], on_missing='warn')

# computing power band features
print('computing power band features')
features_power = []
for ep in tqdm(range(len(epochs_ec))): # get features from each epoch and save in a list
    features_power.append(eeg_power_band(epochs_ec[ep]))
features_power_array = np.concatenate(features_power) # shape of (10332, 310) (nep * (n_chs * 5))
fname = '/Users/payamsadeghishabestari/eeg_data/Regensburg/features_tinnoice_power_array.npy'
np.save(file=fname, arr=features_power_array)

# computing fooof aperiodic parameter
ap_params = []
for ep in tqdm(range(len(epochs_ec))):
    psds, freqs = epochs_ec[ep].compute_psd(method='multitaper', picks='eeg', fmin=0.5, fmax=40, verbose=False).get_data(return_freqs=True)
    fg = FOOOFGroup(aperiodic_mode='fixed')
    params = []
    for ep_i in tqdm(range(len(epochs_ec[ep]))):
        fg.fit(freqs, psds[ep_i], freq_range=[0.5, 40])
        params.append(fg.get_params('aperiodic_params'))
    ap_params.append(params)
features_ap_fooof = np.concatenate(ap_params) # (n_ep * n_ch * 2) --> (n_ep * (n_ch * 2))
fname = '/Users/payamsadeghishabestari/eeg_data/Regensburg/features_tinnoice_ap_fooof.npy'
np.save(file=fname, arr=features_ap_fooof)

3. e. Feature extraction (connectivity = 20 * 20 * 4 = 800)

In [ ]:
# making equal channel size
for ep in epochs_ec:
    ep.drop_channels(['HRli', 'HRre'], on_missing='warn')

fs_dir = fetch_fsaverage(verbose=False)
trans = "fsaverage"  
src = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
bem = op.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")
method = "dSPM"
snr = 3.0
lambda2 = 1.0 / snr**2
sfreq = 250
counter = 0
# selecting important brain labels
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc') # 69 labels

# imp_brain_label_list = ['parahippocampal-lh', 'parahippocampal-rh', 'transversetemporal-lh', 'transversetemporal-rh',
#                         'pericalcarine-lh', 'pericalcarine-rh', 'temporalpole-lh', 'temporalpole-rh',
#                         'medialorbitofrontal-lh', 'medialorbitofrontal-rh', 'lateraloccipital-lh', 'lateraloccipital-rh',
#                         'inferiortemporal-lh', 'inferiortemporal-rh', 'superiortemporal-lh', 'superiortemporal-rh',
#                         'supramarginal-lh', 'supramarginal-rh', 'frontalpole-lh', 'frontalpole-rh', 'parsopercularis-lh', 'parsopercularis-rh',
#                         'posteriorcingulate-lh', 'posteriorcingulate-rh', 'superiorparietal-lh', 'superiorparietal-rh',
#                         'rostralmiddlefrontal-lh', 'rostralmiddlefrontal-rh', 'inferiortemporal-lh', 'inferiortemporal-rh',
#                         'caudalmiddlefrontal-lh', 'caudalmiddlefrontal-rh', 'parstriangularis-lh', 'parstriangularis-rh',
#                         'lateraloccipital-lh', 'lateraloccipital-rh', 'postcentral-lh', 'postcentral-rh'] # RGB40

# imp_brain_label_list = ['parahippocampal-lh', 'parahippocampal-rh', 'paracentral-lh', 'paracentral-rh',
#                         'temporalpole-lh', 'temporalpole-rh', 'superiorfrontal-lh', 'superiorfrontal-rh',
#                         'superiorparietal-lh', 'superiorparietal-rh', 'transversetemporal-lh', 'transversetemporal-rh',
#                         'rostralanteriorcingulate-lh', 'rostralanteriorcingulate-rh', 'insula-lh', 'insula-rh',
#                         'posteriorcingulate-lh', 'posteriorcingulate-rh', 'precentral-lh', 'precentral-rh',
#                         'middletemporal-lh', 'middletemporal-rh', 'frontalpole-lh', 'frontalpole-rh',
#                         'supramarginal-lh', 'supramarginal-rh'] 

imp_brain_label_list = ['superiorparietal-lh', 'superiorparietal-rh',
            'superiortemporal-lh', 'superiortemporal-rh',
            'transversetemporal-lh', 'transversetemporal-rh']

imp_lb_idxs = []
for im_l in imp_brain_label_list:
    for l_idx, l_id in enumerate(brain_labels):
        if l_id.name == im_l:
            imp_lb_idxs.append(l_idx)
sel_brain_labels = list(np.array(brain_labels)[imp_lb_idxs])

# looping over subjects to compute connectivity
for ep in tqdm(epochs_ec[:]):
    # initiating
    ep.set_eeg_reference('average', projection=True)
    ep.apply_proj()
    fwd = mne.make_forward_solution(ep.info, trans=trans, src=src, bem=bem, eeg=True, mindist=5.0, verbose=False)
    # noise_cov = mne.compute_covariance(ep, tmax=0.0, method=["shrunk", "empirical"], rank=None, verbose=False)
    noise_cov = mne.make_ad_hoc_cov(ep.info, std=None, verbose=False)
    inverse_operator = make_inverse_operator(ep.info, fwd, noise_cov, loose=0.2, depth=0.8, verbose=False)
    
    # computing source estimate over all surface
    stc = apply_inverse_epochs(ep, inverse_operator, lambda2=lambda2,
                                                method=method, label=None,
                                                return_generator=False, verbose=False)
    # project it from whole surface to labels
    label_ts = mne.extract_label_time_course(stc, sel_brain_labels, inverse_operator['src'], mode='mean',
                                            allow_empty=True, return_generator=False, verbose=False)

    # # compute connectivity
    # freqs = np.linspace(4.5, 8.5, 5)
    # con = spectral_connectivity_time(np.array(label_ts), freqs=freqs, method=['coh', 'wpli'], average=False, sfreq=sfreq,
    #                                 fmin=freqs[0], fmax=freqs[-1], mode='multitaper', n_cycles=7,
    #                                 faverage=True, n_jobs=1, verbose=False)
    # conmat_coh = con[0].get_data(output='dense')[:, :, :, 0]
    # conmat_wpli = con[1].get_data(output='dense')[:, :, :, 0]
    # fname_coh = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/20_labels/con_coh_theta_{counter}.npy'
    # fname_wpli = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/20_labels/con_wpli_theta_{counter}.npy'
    # np.save(fname_coh, conmat_coh)
    # np.save(fname_wpli, conmat_wpli)
    
    # freqs = np.linspace(15.5, 30, 5)
    # con = spectral_connectivity_time(np.array(label_ts), freqs=freqs, method=['coh', 'wpli'], average=False, sfreq=sfreq,
    #                                 fmin=freqs[0], fmax=freqs[-1], mode='cwt_morlet', n_cycles=7,
    #                                 faverage=True, n_jobs=1, verbose=False)
    # conmat_coh = con[0].get_data(output='dense')[:, :, :, 0]
    # conmat_wpli = con[1].get_data(output='dense')[:, :, :, 0]
    # fname_coh = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/20_labels/con_coh_beta_{counter}.npy'
    # fname_wpli = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/20_labels/con_wpli_beta_{counter}.npy'
    # np.save(fname_coh, conmat_coh)
    # np.save(fname_wpli, conmat_wpli)

    freqs = np.linspace(8, 13, 5)
    con = spectral_connectivity_time(np.array(label_ts), freqs=freqs, method='coh', average=False, sfreq=sfreq,
                                    fmin=freqs[0], fmax=freqs[-1], mode='cwt_morlet', n_cycles=7,
                                    faverage=True, n_jobs=1, verbose=False)
    conmat_coh = con.get_data(output='dense')[:, :, :, 0]
    fname_coh = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/within_aud_network/con_coh_alpha_{counter}.npy'
    np.save(fname_coh, conmat_coh)
    
    
    freqs = np.linspace(30, 80, 5)
    con = spectral_connectivity_time(np.array(label_ts), freqs=freqs, method='coh', average=False, sfreq=sfreq,
                                    fmin=freqs[0], fmax=freqs[-1], mode='cwt_morlet', n_cycles=7,
                                    faverage=True, n_jobs=1, verbose=False)
    conmat_coh = con.get_data(output='dense')[:, :, :, 0]
    fname_coh = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/within_aud_network/con_coh_gamma_{counter}.npy'
    np.save(fname_coh, conmat_coh)
    

    # freqs = np.linspace(8.5, 13.5, 5)
    # con = spectral_connectivity_time(np.array(label_ts), freqs=freqs, method=['coh', 'wpli'], average=False, sfreq=sfreq,
    #                                 fmin=freqs[0], fmax=freqs[-1], mode='cwt_morlet', n_cycles=7,
    #                                 faverage=True, n_jobs=1, verbose=False)
    # conmat_coh = con[0].get_data(output='dense')[:, :, :, 0]
    # conmat_wpli = con[1].get_data(output='dense')[:, :, :, 0]
    # fname_coh = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/20_labels/con_coh_alpha_{counter}.npy'
    # fname_wpli = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/20_labels/con_wpli_alpha_{counter}.npy'
    # np.save(fname_coh, conmat_coh)
    # np.save(fname_wpli, conmat_wpli)
    
    counter += 1

In [ ]:
folder_path = '/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/within_aud_network'  
alpha_files = []; gamma_files = []
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    files_in_folder = sorted(os.listdir(folder_path), key=lambda x: os.path.getmtime(os.path.join(folder_path, x)))
    for filename in files_in_folder:
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and file_path.endswith(".npy"):
            if 'alpha' in file_path:
                alpha_files.append(file_path)
            if 'gamma' in file_path:
                gamma_files.append(file_path)

In [ ]:
fsss_alpha = []
for file in alpha_files[:]:
    x = np.load(file, allow_pickle=True)
    fss = []
    for k in x:
        fs = []
        for i, j in itertools.product(range(k.shape[0]), range(x.shape[1])):
            if j < i:
                fs.append(k[i][j])
        fss.append(fs)
    fsss_alpha.append(fss)

con_alpha_f_array = np.array(fsss_alpha[0])
for i in fsss_alpha[1:]:
    con_alpha_f_array = np.concatenate((con_alpha_f_array, i), axis=0)

fsss_gamma = []
for file in gamma_files[:]:
    x = np.load(file, allow_pickle=True)
    fss = []
    for k in x:
        fs = []
        for i, j in itertools.product(range(k.shape[0]), range(x.shape[1])):
            if j < i:
                fs.append(k[i][j])
        fss.append(fs)
    fsss_gamma.append(fss)

con_gamma_f_array = np.array(fsss_gamma[0])
for i in fsss_gamma[1:]:
    con_gamma_f_array = np.concatenate((con_gamma_f_array, i), axis=0)

con_34lbs_f_array = np.concatenate((con_alpha_f_array, con_gamma_f_array), axis=1)

3. f. Feature extraction (Power in labels using beamformer= 68*5 = 340)

In [ ]:
# making equal channel size
for ep in epochs_ec:
    ep.drop_channels(['HRli', 'HRre'], on_missing='warn')

# Compute PSD in brain labels of inverse solution from single epochs
fs_dir = fetch_fsaverage(verbose=False)
trans = "fsaverage"  
src = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
bem = op.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc') # 69 labels
method = "dSPM"
snr = 3.0
lambda2 = 1.0 / snr**2
counter = 0
for ep in tqdm(epochs_ec[:]):
    ep.set_eeg_reference('average', projection=True)
    fwd = mne.make_forward_solution(ep.info, trans=trans, src=src, bem=bem, eeg=True, mindist=5.0, verbose=False)
    data_cov = mne.compute_covariance(ep, tmin=0.01, tmax=2, method=["shrunk", "empirical"])
    noise_cov = mne.compute_covariance(ep, tmax=0.0, method=["shrunk", "empirical"], rank=None, verbose=False)
    csd_signal = mne.time_frequency.csd_multitaper(ep, fmin=30, fmax=80, verbose=False).mean()
    filters = mne.beamformer.make_dics(ep.info, fwd, csd_signal, reg=0.05, pick_ori="max-power", depth=1.0,
                                        inversion="single", weight_norm=None, real_filter=True, verbose=False)
    #stc_power = mne.beamformer.apply_dics_epochs(ep, filters, return_generator=False, verbose=False)
    stc_power = mne.beamformer.apply_csd(csd_signal, filters, return_generator=False, verbose=False)

    label_ts = []
    for bl in brain_labels[:-1]:
        label_ts.append(mne.extract_label_time_course(stc_power, bl, fwd['src'], mode='mean_flip',
                                                allow_empty=True, return_generator=False, verbose=False))
    file = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/beamformer/dics_averaged/label_ts_gamma_{counter}'
    np.save(file=file, arr=np.array(label_ts))      
    counter += 1

4. a. Importing Saved features (saving new features)

In [ ]:
delta_files = []; theta_files = []; alpha_files = []; beta_files = []; gamma_files = []
folder_path = '/Users/payamsadeghishabestari/eeg_data/Regensburg/beamformer/dics_averaged'  
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    files_in_folder = sorted(os.listdir(folder_path), key=lambda x: os.path.getmtime(os.path.join(folder_path, x)))
    for filename in files_in_folder:
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and file_path.endswith(".npy"):
            if 'delta' in file_path:
                delta_files.append(file_path)
            if 'theta' in file_path:
                theta_files.append(file_path)
            if 'alpha' in file_path:
                alpha_files.append(file_path)
            if 'beta' in file_path:
                beta_files.append(file_path)
            if 'gamma' in file_path:
                gamma_files.append(file_path)

In [ ]:
d_arrs = []; t_arrs = []; a_arrs = []; b_arrs = []; g_arrs = []
for f_d, f_t, f_a, f_b, f_g in zip(delta_files, theta_files, alpha_files, beta_files, gamma_files): 
    d_arrs.append(np.mean(np.squeeze(np.load(f_d, allow_pickle=True)), axis=2))
    t_arrs.append(np.mean(np.squeeze(np.load(f_t, allow_pickle=True)), axis=2))
    a_arrs.append(np.mean(np.squeeze(np.load(f_a, allow_pickle=True)), axis=2))
    b_arrs.append(np.mean(np.squeeze(np.load(f_b, allow_pickle=True)), axis=2))
    g_arrs.append(np.mean(np.squeeze(np.load(f_g, allow_pickle=True)), axis=2))
    

In [ ]:
d_arrs_c = d_arrs[0]
t_arrs_c = t_arrs[0]
a_arrs_c = a_arrs[0]
b_arrs_c = b_arrs[0]
g_arrs_c = g_arrs[0]

for d_arr, t_arr, a_arr, b_arr, g_arr in zip(d_arrs[1:], t_arrs[1:], a_arrs[1:], b_arrs[1:], g_arrs[1:]):
    d_arrs_c = np.concatenate((d_arrs_c, d_arr), axis=1)
    t_arrs_c = np.concatenate((t_arrs_c, t_arr), axis=1)
    a_arrs_c = np.concatenate((a_arrs_c, a_arr), axis=1)
    b_arrs_c = np.concatenate((b_arrs_c, b_arr), axis=1)
    g_arrs_c = np.concatenate((g_arrs_c, g_arr), axis=1)


4. b. Importing Saved features (loading new features)

In [ ]:
# loading the features
features_power_array = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/features_power_array.npy', allow_pickle=True)
features_ent_array = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/features_ent_array.npy', allow_pickle=True)
features_ap_fooof = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/features_ap_fooof.npy', allow_pickle=True) # needs reshaping
features_bl_array = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/power_brain_features.npy', allow_pickle=True)
features_ent_array = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/features_ent_sh_array.npy', allow_pickle=True)
features_coh = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/concatenated/alpha_gamma_coh.npy', allow_pickle=True)
features_wpli = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/concatenated/alpha_gamma_wpli.npy', allow_pickle=True)
features_bmf = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/features_bmf.npy', allow_pickle=True)
features_bl_splitted = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/power_brain_features_splitted.npy', allow_pickle=True)
features_teas_bl_power = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/power_brain_features_teas.npy', allow_pickle=True)
features_teas_bl_power = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/power_brain_features_teas_v3.npy', allow_pickle=True)
features_tinnoice_bl_power = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/power_brain_features_tinnoice.npy', allow_pickle=True)
features_rgb_bl_con = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/con_34lbs_features.npy', allow_pickle=True)
features_teas_bl_con = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/con_teas_features.npy', allow_pickle=True)
features_bl_rgb_mean_array = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/power_brain_features_rgb_mean.npy', allow_pickle=True)
features_bl_teas_mean_array = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/power_brain_features_teas_mean.npy', allow_pickle=True)
features_con_rgb_mean = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_connectivity_rgb_mean.npy', allow_pickle=True)
features_con_teas_mean = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_connectivity_teas_mean.npy', allow_pickle=True)
features_con_rgb_network = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_network_connectivity_rgb.npy', allow_pickle=True)
features_con_within_aud = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_connectivity_within_aud.npy', allow_pickle=True)

features_con_teas_network = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_network_connectivity_teas.npy', allow_pickle=True)
features_con_within_aud_teas = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_connectivity_within_aud_teas.npy', allow_pickle=True)

features_power_teas = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_teas_power_array.npy', allow_pickle=True)
features_ent_teas = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_teas_ent.npy', allow_pickle=True)
features_ap_teas = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_teas_ap_fooof.npy', allow_pickle=True)

# features_power_teas = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_teas_new_power_array.npy', allow_pickle=True)
# features_ent_teas = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_teas_new_ent.npy', allow_pickle=True)
# features_ap_teas = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_teas_new_ap_fooof.npy', allow_pickle=True)

# features_power_teas = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_tinnoice_power_array.npy', allow_pickle=True)
# features_ent_teas = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_tinnoice_ent.npy', allow_pickle=True)
# features_ap_teas = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/features_tinnoice_ap_fooof.npy', allow_pickle=True)

# reshaping ap_fooof features
features_ap_fooof_t = np.transpose(features_ap_fooof, axes=[0,2,1])
features_ap_fooof_array = features_ap_fooof_t.reshape(features_ap_fooof_t.shape[0], -1)

# concatenating features
features_array = np.concatenate((features_power_array, features_ent_array, features_ap_fooof_array), axis=1)
# features_ap_teas = features_ap_teas.reshape(features_ap_teas.shape[0], -1)
# features_array = np.concatenate((features_power_teas, features_ent_teas, features_ap_teas), axis=1)

# features_array = np.concatenate((features_con_teas_network, features_con_within_aud_teas), axis=1)


#features_array.shape

In [ ]:
# features_bl_rgb_mean_array = np.load('/Users/payamsadeghishabestari/eeg_data/Regensburg/power_brain_features_rgb_mean.npy', allow_pickle=True)
# features_array = features_bl_teas_mean_array
features_array.shape

5. Features correlation and distributions

In [ ]:
# here we check which labels have more correlations (needs interpretation)
# histogram
df = pd.DataFrame(features_array)
corr = df.corr(method='pearson')
thr = 0.9
n_reps = np.zeros(features_array.shape[1])
tupels = []
for f1, f2 in product(range(features_array.shape[1]), range(features_array.shape[1])):
    if f1 < f2 and corr[f1][f2] > thr:
        tupels.append((f1, f2))
        n_reps[f1] += 1

# correlation matrix
plt.figure(figsize=(7,5))
plt.matshow(corr, cmap='coolwarm', fignum=1, vmin=-1, vmax=1)
plt.colorbar()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, ax = plt.subplots(1,1, figsize=(6,6))
# ax.matshow(corr, cmap='coolwarm')
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.15)
im = ax.imshow(corr, cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(im, cax=cax, orientation='vertical')

In [ ]:
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1]
# for pair in np.array(tupels)[77:90] - 2 * 68: # alpha
#     print(f'{brain_labels[pair[0]].name} and {brain_labels[pair[1]].name} are merged.')

for pair in np.array(tupels)[-3:] - 4 * 68: # gamma
    print(f'{brain_labels[pair[0]].name} and {brain_labels[pair[1]].name} are merged.')

In [ ]:
# removing features with high correlation
thr = 0.9
counter = 0
rmv_idxs = []
delta_drop, theta_drop, l_alpha_drop, u_alpha_drop, m_beta_drop, u_beta_drop, l_gamma_drop, u_gamma_drop = (0, 0, 0, 0, 0, 0, 0, 0)
for f1, f2 in product(range(features_array.shape[1]), range(features_array.shape[1])):
    if f1 < f2 and corr[f1][f2] > thr:
        counter +=1
        rmv_idxs.append(f1)
        if 0*68 <= f1 < 1*68:
            delta_drop += 1
        if 1*68 <= f1 < 2*68:
            theta_drop += 1
        if 2*68 <= f1 < 3*68:
            l_alpha_drop += 1
        if 3*68 <= f1 < 4*68:
            u_alpha_drop += 1
        if 4*68 <= f1 < 5*68:
            m_beta_drop += 1
        if 5*68 <= f1 < 6*68:
            u_beta_drop += 1
        if 6*68 <= f1 < 7*68:
            l_gamma_drop += 1
        if 7*68 <= f1 < 8*68:
            u_gamma_drop += 1

# df.drop(df.columns[rmv_idxs], axis=1, inplace=True) 
keep_idxs = [i for i in range(features_array.shape[1]) if i not in rmv_idxs]
features_array_after_corr = features_array[:,keep_idxs]
features_array_after_corr.shape

In [ ]:
# plotting the barplots
fig, ax = plt.subplots(1, 1, figsize=(16,4))
ax.bar(x=['delta', 'theta', 'lower alpha', 'upper alpha', 'middle beta',
            'upper beta', 'lower gamma', 'upper gamma'], 
            height=[delta_drop, theta_drop, l_alpha_drop, u_alpha_drop,
            m_beta_drop, u_beta_drop, l_gamma_drop, u_gamma_drop])
ax.grid(True)

6. Importing Classification Packages

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRFClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

6. a. Training with various CLFs (Random Forest)

In [ ]:
# random forest
n_sel_features = 100
model = RandomForestClassifier(n_estimators=100)
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=None)
mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1

n_scores = cross_val_score(model, X_train, y_train, groups = group_train ,scoring='accuracy', cv=10, n_jobs=-1)
model.fit(X_train, y_train)

# reporting and features overview
importance = model.feature_importances_
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')

# re-training with selected features
selected_features_idx = np.argsort(importance)[-n_sel_features:]
new_features_array = features_array_after_corr[:,selected_features_idx]
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(new_features_array,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=None)

mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1

n_scores = cross_val_score(model, X_train, y_train, groups = group_train ,scoring='accuracy', cv=10, n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f'accuracy of the model checked with test data with eliminating features: {accuracy}')

In [ ]:
print(report)

In [ ]:
# save for validation set
fname = '/Users/payamsadeghishabestari/Prime/connectivity_teas_idxs.npy'
np.save(fname, np.array(keep_idxs)[selected_features_idx])
model_filename = '/Users/payamsadeghishabestari/Prime/connectivity_teas_model.pkl'
joblib.dump(model, model_filename)

In [ ]:
# now checking the importances
#importance = model.feature_importances_
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc') # 69 labels
delta_imps, theta_imps, alpha_imps, beta_imps, gamma_imps = (0, 0, 0, 0, 0)
teas_alpha_names = []; teas_alpha_imps= []; teas_gamma_names = []; teas_gamma_imps= []; teas_theta_imps = []; teas_delta_imps = []; teas_beta_imps = []
# for f_idx, im in zip(np.array(keep_idxs)[selected_features_idx][:], np.sort(importance)[-n_sel_features:][:]):
#     if 0*68 <= f_idx < 1*68:
#         delta_imps += im
#         # print(f'alpha labels {brain_labels[f_idx - 0 * 68].name} with importance {im}')
#         teas_delta_imps.append(im)
#     if 1*68 <= f_idx < 2*68:
#         theta_imps += im
#         # print(f'alpha labels {brain_labels[f_idx - 1 * 68].name} with importance {im}')
#         teas_theta_imps.append(im)
#     if 2*68 <= f_idx < 3*68:
#         alpha_imps += im
#         # print(f'alpha labels {brain_labels[f_idx - 2 * 68].name} with importance {im}')
#         teas_alpha_names.append(brain_labels[f_idx - 2 * 68].name)
#         teas_alpha_imps.append(im)
#     if 3*68 <= f_idx < 4*68:
#         beta_imps += im
#         teas_beta_imps.append(im)
#     if 4*68 <= f_idx < 5*68:
#         gamma_imps += im
#         # print(f'gamma labels {brain_labels[f_idx - 4 * 68].name} with importance {im}')
#         teas_gamma_names.append(brain_labels[f_idx - 4 * 68].name)
#         teas_gamma_imps.append(im)

delta_imps, theta_imps, alpha_imps, beta_imps, gamma_imps = (0, 0, 0, 0, 0)
ch_names = epochs_ec[0].info['ch_names']
for f_idx, im in zip(np.array(keep_idxs)[selected_features_idx], np.sort(importance)[-n_sel_features:]):
    if 0*64 <= f_idx < 1*64:
        delta_imps += im
    if 1*64 <= f_idx < 2*64:
        theta_imps += im
    if 2*64 <= f_idx < 3*64:
        alpha_imps += im
        # print(f'alpha labels {ch_names[f_idx - 2 * 64]} with importance {im}')
        #print(f_idx - 2 * 62)
    if 3*64 <= f_idx < 4*64:
        beta_imps += im
    if 4*64 <= f_idx < 5*64:
        gamma_imps += im
        print(f'gamma labels {ch_names[f_idx - 4 * 64]} with importance {im}')
        # print(f_idx - 4 * 64)

In [ ]:
# check the sns nodel with validation data
model_fname = '/Users/payamsadeghishabestari/Prime/source_space_model.pkl'
model = joblib.load(model_fname)
sel_idxs_fname = '/Users/payamsadeghishabestari/Prime/source_space_f_idxs.npy'
sel_idxs = np.load(sel_idxs_fname, allow_pickle=True)
y_pred = model.predict(features_array[:, sel_idxs])
accuracy_sns = accuracy_score(label_array, y_pred)

In [ ]:
accuracy_sns

In [ ]:
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc') # 69 labels
for f_idx, f_imp in zip(np.array(keep_idxs)[selected_features_idx][:], np.sort(importance)[-n_sel_features:][:]):
    #if 0*68 <= f_idx < 1*68:
    #    print(f'delta with labels {brain_labels[f_idx].name}')
    #if 1*68 <= f_idx < 2*68:
    #    print(f'theta with labels {brain_labels[f_idx - 1 * 68].name}')
    # if 2*68 <= f_idx < 3*68:
    #     print(f'low alpha with labels {brain_labels[f_idx - 2 * 68].name} with importance {f_imp}')
    # if 3*68 <= f_idx < 4*68:
    #     print(f'high alpha with labels {brain_labels[f_idx - 3 * 68].name} with importance {f_imp}')
    #if 4*68 <= f_idx < 5*68:
    #    print(f'middle beta labels {brain_labels[f_idx - 4 * 68].name}')
    #if 5*68 <= f_idx < 6*68:
    #    print(f'higher beta labels {brain_labels[f_idx - 5 * 68].name}')
    #if 6*68 <= f_idx < 7*68:
    #    print(f'low gamma labels {brain_labels[f_idx - 6 * 68].name} with importance {f_imp}')
    #if 7*68 <= f_idx < 8*68:
    #    print(f'high gamma labels {brain_labels[f_idx - 7 * 68].name} with importance {f_imp}')

6. b. Training with various CLFs (XGBRF)

In [ ]:
model = XGBRFClassifier(n_estimators=100, subsample=0.9, colsample_bynode=0.2,
                            objective = 'binary:logistic', eta=0.001)
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=42)
n_scores = cross_val_score(model, X_train, y_train, groups = group_train ,scoring='accuracy', cv=10, n_jobs=-1)
model.fit(X_train, y_train)

# reporting and features overview
importance = model.feature_importances_
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')

# keeping only features with higher importance and labeling them
Labels = ['delta_power', 'theta_power', 'alpha_power', 'beta_power', 'gamma_power',
            'delta_ent', 'theta_ent', 'alpha_ent', 'beta_ent', 'gamma_ent',
            'aperiodic_offset', 'aperiodic_exponent',
            'delta_label_power', 'theta_label_power', 'alpha_label_power', 'beta_label_power', 'gamma_label_power']
features_label = []
selected_features_idx = np.argsort(importance)[-n_sel_features:] # selecting only 100 features with highest importance (almost equal to importance > 0.005)
for idx in selected_features_idx:
    for l_idx in range(len(Labels)):
        if keep_idxs[idx] < 12 * 62:
            if l_idx * 62 <= keep_idxs[idx] < (l_idx + 1) * 62:
                features_label.append(Labels[l_idx])
        if 12 * 62 <= keep_idxs[idx]:
            if l_idx * 68 <= keep_idxs[idx] < (l_idx + 1) * 68:
                features_label.append(Labels[l_idx])
    
for l_idx in range(len(Labels)):
    print(f'{Labels[l_idx]} accounts for {int(len(np.where(np.array(features_label) == Labels[l_idx])[0]) / n_sel_features * 100)} % of selected features')
    
# re-training with selected features
new_features_array = features_array_after_corr[:,selected_features_idx]
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(new_features_array,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=42)
n_scores = cross_val_score(model, X_train, y_train, groups = group_train ,scoring='accuracy', cv=10, n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data with eliminating features: {accuracy}')

# which channels and which labels
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
sensors_labels = []
for idx in np.array(keep_idxs)[selected_features_idx]:
    for l_idx in range(len(Labels)):
        if idx < 5 * 62:
            if l_idx * 62 <= idx < (l_idx + 1) * 62:
                sensors_labels.append((Labels[l_idx], epochs_ec[0].info['ch_names'][idx - l_idx * 62]))
        if 12 * 62 <= idx:
            if l_idx * 68 <= idx < (l_idx + 1) * 68:
                sensors_labels.append((Labels[l_idx], brain_labels[idx - l_idx * 68].name))

6. c. Training with various CLFs (Linear SVM)

In [ ]:
model = SVC(C=1.0, random_state=42, kernel='linear')
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=42)

mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1
model.fit(X_train, y_train)

# reporting and features overview
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')

6. d. Training with various CLFs (CNN)

In [ ]:
model = MLPClassifier(activation='relu', alpha=1e-5, shuffle=True, 
                    hidden_layer_sizes=(100,), random_state=42)
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle=True, random_state=42)
mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1
model.fit(X_train, y_train)

# reporting and features overview
#importance = model.feature_importances_
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')

# keeping only features with higher importance and labeling them
Labels = ['delta_power', 'theta_power', 'alpha_power', 'beta_power', 'gamma_power',
        'delta_ent', 'theta_ent', 'alpha_ent', 'beta_ent', 'gamma_ent',
            'aperiodic_offset', 'aperiodic_exponent',
            'delta_label_power', 'theta_label_power', 'alpha_label_power', 'beta_label_power', 'gamma_label_power']
features_label = []
selected_features_idx = np.argsort(importance)[-n_sel_features:] # selecting only 100 features with highest importance (almost equal to importance > 0.005)
for idx in selected_features_idx:
    for l_idx in range(len(Labels)):
        if keep_idxs[idx] < 12 * 62:
            if l_idx * 62 <= keep_idxs[idx] < (l_idx + 1) * 62:
                features_label.append(Labels[l_idx])
        if 12 * 62 <= keep_idxs[idx]:
            if l_idx * 68 <= keep_idxs[idx] < (l_idx + 1) * 68:
                features_label.append(Labels[l_idx])

6. e. Training with various CLFs (K-NN)

In [ ]:
model = KNeighborsClassifier()
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=42)
mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1
model.fit(X_train, y_train)

# reporting and features overview
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')

6. f. Training with various CLFs (GP)

In [ ]:
model = GaussianProcessClassifier(1.0 * RBF(1.0), random_state=42)
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=42)
mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1
model.fit(X_train, y_train)

# reporting and features overview
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')

6. g. Training with various CLFs (RBF SVM)

In [ ]:
model = SVC(random_state=42)
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=42)
mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1
model.fit(X_train, y_train)

# reporting and features overview
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')

6. h. Training with various CLFs (Decision Tree)

In [ ]:
model = DecisionTreeClassifier(random_state=42)
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=42)
mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1
model.fit(X_train, y_train)

# reporting and features overview
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')


6. i. Training with various CLFs (Naive Bayes)

In [ ]:
model = GaussianNB()
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=42)
mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1
model.fit(X_train, y_train)

# reporting and features overview
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')

6. j. Training with various CLFs (QDA)

In [ ]:
model = QuadraticDiscriminantAnalysis()
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=42)
mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1
model.fit(X_train, y_train)

# reporting and features overview
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')

6. k. Training with various CLFs (penalizing with elastic net)

In [ ]:
model = LogisticRegression(penalty='elasticnet', solver='saga', C=1,
                        l1_ratio=0.5, max_iter=10000, random_state=42)
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=0)
mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1

model.fit(X_train, y_train)

# reporting and features overview
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')

Checking Importance of features in Random Forest

In [ ]:
# all_delta_imps = []
# all_theta_imps = []
# all_alpha_imps = []
# all_beta_imps = []
# all_gamma_imps = []
# all_delta_ent_imps = []
# all_theta_ent_imps = []
# all_alpha_ent_imps = []
# all_beta_ent_imps = []
# all_gamma_ent_imps = []
# all_ap_1_imps = []
# all_ap_2_imps = []
iteration = 0
while iteration < 1:
    # initial training
    model = RandomForestClassifier(n_estimators=100)
    X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                                label_array, group_array,
                                                                                test_size=0.3, train_size=0.7,
                                                                                shuffle = True, random_state=0)
    n_scores = cross_val_score(model, X_train, y_train, groups = group_train ,scoring='accuracy', cv=10, n_jobs=-1)

    # handling Nan values for random forest
    mask_train = np.isnan(X_train)
    mask_test = np.isnan(X_test)
    nan_indices_train = np.where(mask_train)
    nan_indices_test = np.where(mask_test)
    for i in range(len(nan_indices_train[0])):
        X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
    for i in range(len(nan_indices_test[0])):
        X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1

    # selecting features with highest importance
    model.fit(X_train, y_train)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')
    importances = model.feature_importances_
    n_sel_features = 100
    selected_features_idx = np.argsort(importances)[-n_sel_features:]
    new_features_array = features_array_after_corr[:,selected_features_idx]

    # re-training with selected features
    X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(new_features_array,
                                                                                label_array, group_array,
                                                                                test_size=0.3, train_size=0.7,
                                                                                shuffle = True, random_state=0)
    n_scores = cross_val_score(model, X_train, y_train, groups = group_train ,scoring='accuracy', cv=10, n_jobs=-1)
    # handling Nan values for random forest
    mask_train = np.isnan(X_train)
    mask_test = np.isnan(X_test)
    nan_indices_train = np.where(mask_train)
    nan_indices_test = np.where(mask_test)
    for i in range(len(nan_indices_train[0])):
        X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
    for i in range(len(nan_indices_test[0])):
        X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'accuracy of the model checked with test data with eliminating features: {accuracy}')
    importance = model.feature_importances_

    delta_imps, theta_imps, alpha_imps, beta_imps, gamma_imps = (0, 0, 0, 0, 0)
    delta_ent_imps, theta_ent_imps, alpha_ent_imps, beta_ent_imps, gamma_ent_imps = (0, 0, 0, 0, 0)
    ap_1_imps, ap_2_imps = (0, 0)
    for f_idx, im in zip(np.array(keep_idxs)[selected_features_idx], importance):
        if 0*68 <= f_idx < 1*68:
            delta_imps += im
        if 1*68 <= f_idx < 2*68:
            theta_imps += im
        if 2*68 <= f_idx < 3*68:
            alpha_imps += im
        if 3*68 <= f_idx < 4*68:
            beta_imps += im
        if 4*68 <= f_idx < 5*68:
            gamma_imps += im
        
        # if 0*62 <= f_idx < 1*62:
        #     delta_imps += im
        # if 1*62 <= f_idx < 2*62:
        #     theta_imps += im
        # if 2*62 <= f_idx < 3*62:
        #     alpha_imps += im
        # if 3*62 <= f_idx < 4*62:
        #     beta_imps += im
        # if 4*62 <= f_idx < 5*62:
        #     gamma_imps += im
        # if 5*62 <= f_idx < 6*62:
        #     delta_ent_imps += im
        # if 6*62 <= f_idx < 7*62:
        #     theta_ent_imps += im
        # if 7*62 <= f_idx < 8*62:
        #     alpha_ent_imps += im
        # if 8*62 <= f_idx < 9*62:
        #     beta_ent_imps += im
        # if 9*62 <= f_idx < 10*62:
        #     gamma_ent_imps += im
        # if 10*62 <= f_idx < 11*62:
        #     ap_1_imps += im
        # if 11*62 <= f_idx < 12*62:
        #     ap_2_imps += im

    # all_delta_imps.append(delta_imps)
    # all_theta_imps.append(theta_imps)
    # all_alpha_imps.append(alpha_imps)
    # all_beta_imps.append(beta_imps)
    # all_gamma_imps.append(gamma_imps)
    # all_delta_ent_imps.append(delta_ent_imps)
    # all_theta_ent_imps.append(theta_ent_imps)
    # all_alpha_ent_imps.append(alpha_ent_imps)
    # all_beta_ent_imps.append(beta_ent_imps)
    # all_gamma_ent_imps.append(gamma_ent_imps)
    # all_ap_1_imps.append(ap_1_imps)
    # all_ap_2_imps.append(ap_2_imps)
    iteration += 1

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(19,7))
lists = [all_delta_imps, all_theta_imps, all_alpha_imps, all_beta_imps, all_gamma_imps,
        all_delta_ent_imps, all_theta_ent_imps, all_alpha_ent_imps, all_beta_ent_imps, all_gamma_ent_imps,
        all_ap_1_imps, all_ap_2_imps]
names_list = ['Delta power', 'Theta power', 'Alpha power', 'Beta power', 'Gamma power',
        'Delta entropy', 'Theta entropy', 'Alpha entropy', 'Beta entropy', 'Gamma entropy',
        'Aperiodic offset', 'Aperiodic exponent']

data = pd.DataFrame({'Features': [item for item in names_list for _ in range(20)],
                        'Importance': [val for sublist in lists for val in sublist]})

# Create the violin plot using Seaborn
sns.set(style="whitegrid")
sns.violinplot(x='Features', y='Importance', data=data, bw='scott', palette="Set2", saturation=0.75, linewidth=1, ax=ax)
sns.swarmplot(x='Features', y='Importance', data=data, palette="Set2", linewidth=1, ax=ax, size=3)

ax.legend(fontsize=14, frameon=False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.grid(axis='x', color='k', linestyle='--', linewidth=0.5)

In [ ]:
# plotting the importances by category
fig, ax = plt.subplots(1, 1, figsize=(10,10))
ax.barh(y=['delta_ent', 'theta_ent', 'delta_power',
'beta_ent','theta_power', 'alpha_ent', 'ap_1', 'ap_2', 'beta_power',
'gamma_ent', 'alpha_power', 'gamma_power'], width=[delta_ent_imps, 
theta_ent_imps, delta_imps, beta_ent_imps, theta_imps, alpha_ent_imps,
ap_1_imps, ap_2_imps, beta_imps, gamma_ent_imps, alpha_imps, gamma_imps])
ax.spines['left'].set_visible(False); ax.spines['bottom'].set_visible(False)
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.grid(axis='x', linestyle=':')

Checking Partial Dependency of features and labels

In [ ]:
from sklearn.inspection import partial_dependence
from sklearn.inspection import PartialDependenceDisplay

In [ ]:
# choosing the 10 most important indexes for each class of feature
a_p_idxs = []; g_p_idxs = []; a_e_idxs = []; g_e_idxs = []; ap_of_idxs = []; ap_sl_idxs = []
for idx, i in enumerate(np.array(keep_idxs)[selected_features_idx]):
    if 2*62 <= i < 3*62:
        a_p_idxs.append(idx)
    if 4*62 <= i < 5*62:
        g_p_idxs.append(idx)
    if 7*62 <= i < 8*62:
        a_e_idxs.append(idx)
    if 9*62 <= i < 10*62:
        g_e_idxs.append(idx)
    if 10*62 <= i < 11*62:
        ap_of_idxs.append(idx)
    if 11*62 <= i < 12*62:
        ap_sl_idxs.append(idx) 

In [ ]:
# the 10 most important features (pdp apply)
pds = {'alpha_power_values': [], 'alpha_power_pd': [],
        'gamma_power_values': [], 'gamma_power_pd': [],
        'alpha_ent_values': [], 'alpha_ent_pd': [],
        'gamma_ent_values': [], 'gamma_ent_pd': [],
        'ap_off_values': [], 'ap_off_pd': [],
        'ap_sl_values': [], 'ap_sl_pd': []}

for al, ga in zip(alpha_label_idxs[-10:], gamma_label_idxs[-10:]): 
        pd = partial_dependence(model, np.concatenate((X_train, X_test)), features=[al], kind='average')
        pds['alpha_power_values'].append(pd['values'][0])
        pds['alpha_power_pd'].append(pd['average'][0])
        ##
        pd = partial_dependence(model, np.concatenate((X_train, X_test)), features=[ga], kind='average')
        pds['gamma_power_values'].append(pd['values'][0])
        pds['gamma_power_pd'].append(pd['average'][0])
        ##
        #pd = partial_dependence(model, np.concatenate((X_train, X_test)), features=[a_e_f], kind='average')
        #pds['alpha_ent_values'].append(pd['values'][0])
        #pds['alpha_ent_pd'].append(pd['average'][0])
        ##
        #pd = partial_dependence(model, np.concatenate((X_train, X_test)), features=[g_e_f], kind='average')
        #pds['gamma_ent_values'].append(pd['values'][0])
        #pds['gamma_ent_pd'].append(pd['average'][0])
        ##
        # pd = partial_dependence(model, np.concatenate((X_train, X_test)), features=[ap_of], kind='average')
        # pds['ap_off_values'].append(pd['values'][0])
        # pds['ap_off_pd'].append(pd['average'][0])
        # pd = partial_dependence(model, np.concatenate((X_train, X_test)), features=[ap_sl], kind='average')
        # pds['ap_sl_values'].append(pd['values'][0])
        # pds['ap_sl_pd'].append(pd['average'][0])

In [ ]:
lb_names = ['frontalpole-lh', 'supramarginal-rh', 'superiortemporal-rh', 'inferiortemporal-lh', 'lateraloccipital-rh', 'medialorbitofrontal-rh'
, 'temporalpole-rh', 'pericalcarine-lh', 'transversetemporal-rh', 'parahippocampal-lh']
fig, ax = plt.subplots(1, 1, figsize=(6,6))
for i, lb_name in zip(range(10), lb_names):
    ax.plot(pds['gamma_power_values'][i], pds['gamma_power_pd'][i], label=lb_name)
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.legend()
ax.grid(True)

Checking the locations

In [ ]:
# choosing the 10 most important indexes for each class of feature
a_p_idxs = []; g_p_idxs = []; a_e_idxs = []; g_e_idxs = []; ap_of_idxs = []; ap_sl_idxs = []
for idx, i in enumerate(np.array(keep_idxs)[selected_features_idx]):
    if 2*62 <= i < 3*62:
        a_p_idxs.append(idx)
    if 4*62 <= i < 5*62:
        g_p_idxs.append(idx)
    if 7*62 <= i < 8*62:
        a_e_idxs.append(idx)
    if 9*62 <= i < 10*62:
        g_e_idxs.append(idx)
    if 10*62 <= i < 11*62:
        ap_of_idxs.append(idx)
    if 11*62 <= i < 12*62:
        ap_sl_idxs.append(idx) 

alpha_label_idxs = []; gamma_label_idxs = []
for idx, i in enumerate(np.array(keep_idxs)[selected_features_idx]):
    if 2*68 <= i < 3*68:
        alpha_label_idxs.append(idx)
    if 4*68 <= i < 5*68:
        gamma_label_idxs.append(idx)
yy = []
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 68 labels
for label in gamma_label_idxs:
    lb_idx = np.array(keep_idxs)[selected_features_idx][label] - 4 * 68 # for gamma 4 * 62
    yy.append(brain_labels[lb_idx].name)


In [ ]:
raw_fname = '/Users/payamsadeghishabestari/eeg_data/Regensburg/transfer_147155_files_ac07b771/aaeeg_rsEEG/rd7p_rest.vhdr'
raw_rs = mne.io.read_raw_brainvision(vhdr_fname=raw_fname, preload=True, verbose=False)
new_ch_names = raw_rs.info['ch_names'].copy()
new_ch_names[58] = 'Fpz'
mapping = dict(zip(raw_rs.info['ch_names'], new_ch_names))
mne.rename_channels(raw_rs.info, mapping)
raw_rs.set_montage('standard_1020', verbose=False, on_missing='warn')
ch_names = raw_rs.info['ch_names']

In [ ]:
raw_rs.info['bads'] = []
for ch_i in g_p_idxs:
    ch_idx = np.array(keep_idxs)[selected_features_idx][ch_i] - 4 * 62 # for gamma 4 * 62
    raw_rs.info['bads'].append(ch_names[ch_idx])

In [ ]:
raw_rs.info['bads'][-10:]

In [ ]:
raw_rs.info['bads'] = []
raw_rs.plot_sensors(show_names=True, kind='select')

In [ ]:
for idx, i in enumerate(brain_labels):
    if i.name == 'transversetemporal-rh':
        sel_idx = idx
sel_idx = 4 * 68 + sel_idx
np.where(np.array(corr[sel_idx]) > 0.9)[0] - 4 * 68

8. a. Plotting 3D brain with labels

In [ ]:
# plotting brain with colored labels
# initiating the brain
Brain = mne.viz.get_brain_class()
brain_kwargs = dict(alpha=0.15, background="white", cortex="low_contrast", size=(800, 600), views='medial')
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc') # 69 labels
imp_brain_label_list_gamma = ['parahippocampal-lh', 'transversetemporal-rh', 'pericalcarine-lh', 'temporalpole-rh',
                        'medialorbitofrontal-rh', 'lateraloccipital-rh', 'inferiortemporal-lh', 'superiortemporal-rh',
                        'supramarginal-rh', 'frontalpole-lh']
imp_brain_label_list_alpha = ['parsopercularis-rh', 'posteriorcingulate-rh', 'superiorparietal-rh', 'parahippocampal-rh',
                            'rostralmiddlefrontal-lh', 'inferiortemporal-lh', 'caudalmiddlefrontal-rh', 'parstriangularis-rh',
                            'lateraloccipital-lh', 'postcentral-lh']
imp_lb_a_rh_idxs = []; imp_lb_a_lh_idxs = []
imp_lb_g_rh_idxs = []; imp_lb_g_lh_idxs = []

# indexing right/left and alpha/gamma labels
for im_l_a, im_l_g in zip(imp_brain_label_list_alpha, imp_brain_label_list_gamma):
    for l_idx, l_id in enumerate(brain_labels):
        if l_id.name == im_l_a and l_id.name[-2:] == 'rh':
            imp_lb_a_rh_idxs.append(l_idx)
        if l_id.name == im_l_a and l_id.name[-2:] == 'lh':
            imp_lb_a_lh_idxs.append(l_idx)
        if l_id.name == im_l_g and l_id.name[-2:] == 'rh':
            imp_lb_g_rh_idxs.append(l_idx)
        if l_id.name == im_l_g and l_id.name[-2:] == 'lh':
            imp_lb_g_lh_idxs.append(l_idx)

# plotting
brain = Brain("fsaverage", hemi="rh", surf="pial", title='alpha-rh', **brain_kwargs)
for label in list(np.array(brain_labels)[imp_lb_a_rh_idxs]): # alpha-rh
    brain.add_label(label, hemi="rh", color="#1f77b4", borders=False, alpha=0.5)
brain.save_image(filename='/Users/payamsadeghishabestari/Prime/parcel-medial-alpha-rh.png', mode='rgb')

brain = Brain("fsaverage", hemi="lh", surf="pial", title='alpha-lh', **brain_kwargs)
for label in list(np.array(brain_labels)[imp_lb_a_lh_idxs]): # alpha-lh
    brain.add_label(label, hemi="lh", color="#1f77b4", borders=False, alpha=0.5)
brain.save_image(filename='/Users/payamsadeghishabestari/Prime/parcel-medial-alpha-lh.png', mode='rgb')


brain = Brain("fsaverage", hemi="rh", surf="pial", title='gamma-rh', **brain_kwargs)
for label in list(np.array(brain_labels)[imp_lb_g_rh_idxs]): # gamma-rh
    brain.add_label(label, hemi="rh", color="#ff7f0e", borders=False, alpha=0.5)
brain.save_image(filename='/Users/payamsadeghishabestari/Prime/parcel-medial-gamma-rh.png', mode='rgb')


brain = Brain("fsaverage", hemi="lh", surf="pial", title='gamma-lh', **brain_kwargs)
for label in list(np.array(brain_labels)[imp_lb_g_lh_idxs]): # gamma-lh
    brain.add_label(label, hemi="lh", color="#ff7f0e", borders=False, alpha=0.5)
brain.save_image(filename='/Users/payamsadeghishabestari/Prime/parcel-medial-gamma-lh.png', mode='rgb')

In [ ]:
# plotting only parcelations
Brain = mne.viz.get_brain_class()
brain = Brain("fsaverage", hemi="lh", surf="pial", cortex="low_contrast",
    background="white",
    size=(800, 600))
brain.add_annotation("aparc", borders=False, alpha=0.6)
brain.save_image(filename='/Users/payamsadeghishabestari/Prime/parcel-lh.png', mode='rgb')

8. b. Plotting accuracies of different classifiers

In [ ]:
# plotting barplot accuracies
######## single 90 threshold
clfs = ['Random Forest', 'XGBRF', 'QDA', 'Naive Bayes', 'Desicion Tree', 'RBF SVM', 'GP', 'KNN', 'CNN', 'linear SVM']
heights = [98.6, 94.1, 99.3, 67.0, 89.7, 71.2, 84.3, 84.1, 96.2, 93.7]
alphas = [ 0.9, 0.7, 1, 0.1, 0.5, 0.2, 0.4, 0.3, 0.8, 0.6]
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
p = []
for ci in range(len(clfs)):
       ax.barh(y=clfs[ci], width=heights[ci], height=0.4, color='#1f77b4', left=None, edgecolor=None, linewidth=1,
              alpha=alphas[ci])      
ax.grid(axis='x')
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.set_xlim(50, 100)
ax.set_title('threshold 90 (single)')
for i, v in enumerate(heights):
       ax.text(v + 1, i-.15, f'{v}%', color='red', fontfamily='monospace')

######## single 80 threshold
clfs = ['Random Forest', 'XGBRF', 'QDA', 'Naive Bayes', 'Desicion Tree', 'RBF SVM', 'GP', 'KNN', 'CNN', 'linear SVM']
heights = [97.6, 89.1, 96.81, 70.2, 90.4, 68.1, 84.6, 84.7, 96.2, 94.4]
alphas = [ 0.9, 0.7, 1, 0.1, 0.5, 0.2, 0.4, 0.3, 0.8, 0.6]
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
p = []
for ci in range(len(clfs)):
       ax.barh(y=clfs[ci], width=heights[ci], height=0.4, color='#1f77b4', left=None, edgecolor=None, linewidth=1,
              alpha=alphas[ci])
ax.grid(axis='x')
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.set_xlim(50, 100)
ax.set_title('threshold 80 (single)')
for i, v in enumerate(heights):
       ax.text(v + 1, i-.15, f'{v}%', color='red', fontfamily='monospace')

######## single 70 threshold
clfs = ['Random Forest', 'XGBRF', 'QDA', 'Naive Bayes', 'Desicion Tree', 'RBF SVM', 'GP', 'KNN', 'CNN', 'linear SVM']
heights = [96.8, 90.3, 93.9, 77.5, 89.7, 72.9, 85.8, 86.5, 96.4, 95.8]
alphas = [ 0.9, 0.7, 1, 0.1, 0.5, 0.2, 0.4, 0.3, 0.8, 0.6]
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
p = []
for ci in range(len(clfs)):
       ax.barh(y=clfs[ci], width=heights[ci], height=0.4, color='#1f77b4', left=None, edgecolor=None, linewidth=1,
              alpha=alphas[ci])
ax.grid(axis='x')
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.set_xlim(50, 100)
ax.set_title('threshold 70 (single)')
for i, v in enumerate(heights):
       ax.text(v + 1, i-.15, f'{v}%', color='red', fontfamily='monospace')

######## multi class (larger than 90, 90-70, 70-50, less than 50)
clfs = ['Random Forest', 'XGBRF', 'QDA', 'Naive Bayes', 'Desicion Tree', 'RBF SVM', 'GP', 'KNN', 'CNN', 'linear SVM']
heights = [97.0, 89.8, 65.0, 38.1, 85.8, 55.4, 18.6, 78.5, 95.4, 94.8]
alphas = [ 0.9, 0.7, 1, 0.1, 0.5, 0.2, 0.4, 0.3, 0.8, 0.6]
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
p = []
for ci in range(len(clfs)):
       ax.barh(y=clfs[ci], width=heights[ci], height=0.4, color='#1f77b4', left=None, edgecolor=None, linewidth=1,
              alpha=alphas[ci])
ax.grid(axis='x')
ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
ax.set_xlim(50, 100)
ax.set_title('multi class threshold')
for i, v in enumerate(heights):
       ax.text(v + 1, i-.15, f'{v}%', color='red', fontfamily='monospace')

In [ ]:
# plotting spider plots
df = pd.DataFrame({
'group': ['Thr 90','Thr 80','Thr 70', 'Thr 60', 'Thr 50'],
'Random Forest': [98.6, 97.6, 96.8, 96.7, 96.2],
'XGBRF': [94.1, 89.1, 90.3, 92.12, 91.7],
'QDA': [99.3, 96.8, 93.9, 86, 84.5],
'NB': [67.0, 70.2, 77.5, 81.97, 84.7],
'Desicion Tree': [89.7, 90.4, 89.7, 90.67, 92.8],
'RBF SVM': [71.2, 68.1, 72.9, 79.80, 83.7],
'GP': [84.3, 84.6, 85.8, 90.12, 91.7],
'KNN': [84.1, 84.7, 86.5, 91, 92.1],
'CNN': [96.2, 96.2, 96.4, 97.6, 97.7],
'linear SVM': [93.7, 94.4, 95.8, 96.41, 97.6],
})

# df = pd.DataFrame({
# 'group': ['Thr 90', 'Thr 70', 'Thr 50'],
# 'Random Forest': [99, 98, 96],
# 'XGBRF': [91, 86, 93],
# 'QDA': [84, 80, 80],
# 'NB': [57, 73, 82],
# 'Desicion Tree': [85, 90, 91],
# 'RBF SVM': [73, 77, 86],
# 'GP': [85, 87, 88],
# 'KNN': [80, 84, 90],
# 'CNN': [94, 96, 97],
# 'linear SVM': [82, 87, 92],
# })

def make_spider(row, title, color):

    # number of variable
    categories=list(df)[1:]
    N = len(categories)

    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    # Initialise the spider plot
    ax = plt.subplot(1,5,row+1, polar=True)

    # If you want the first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)

    # Draw one axe per variable + add labels labels yet
    # plt.xticks(angles[:-1], categories, color='black', size=10)

    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([50,60,70,80,90], ["50","60","70","80","90"], color="black", size=11)
    plt.ylim(50,100)

    # Ind1
    values=df.loc[row].drop('group').values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, color=color, linewidth=3, linestyle='solid')
    ax.fill(angles, values, color=color, alpha=0.4)

    # Add a title
    plt.title(title, size=11, color=color, y=1.1)

    
# ------- PART 2: Apply the function to all individuals
# initialize the figure
my_dpi=116
plt.figure(figsize=(12, 4), dpi=my_dpi)
 
# Create a color palette:
my_palette = plt.cm.get_cmap("Set2", len(df.index))
 
# Loop to plot
for row in range(0, len(df.index)):
    make_spider( row=row, title=''+df['group'][row], color=my_palette(row))

8. c. Plotting Connectivity circle plots

In [ ]:
# loading labeling
coh_alpha_files = []; coh_gamma_files = []
wpli_alpha_files = []; wpli_gamma_files = []
lb_list_unique = []
for label in label_list:
    lb_list_unique.append(label[0])

# loading and sorting files
folder_path = '/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/rgb_mean_labels'  
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    files_in_folder = sorted(os.listdir(folder_path), key=lambda x: os.path.getmtime(os.path.join(folder_path, x)))
    for filename in files_in_folder:
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and file_path.endswith(".npy"):
            if 'coh_alpha' in file_path:
                coh_alpha_files.append(file_path)
            if 'coh_gamma' in file_path:
                coh_gamma_files.append(file_path)
            if 'wpli_alpha' in file_path:
                wpli_alpha_files.append(file_path)
            if 'wpli_gamma' in file_path:
                wpli_gamma_files.append(file_path)

In [ ]:
# coherence alpha
inhibit_files = []
non_inhibit_files = []
for f_idx, f in enumerate(coh_alpha_files): 
    if lb_list_unique[f_idx] == 0:
        non_inhibit_files.append(f)
    if lb_list_unique[f_idx] == 1:
        inhibit_files.append(f)

con_arrs = []
for arr_file in inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_arrs.append(np.mean(con_arr, axis=0))
coh_inhibit_alpha_avg = np.mean(np.array(con_arrs), axis=0)
con_arrs = []
for arr_file in non_inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_arrs.append(np.mean(con_arr, axis=0))
coh_non_inhibit_alpha_avg = np.mean(np.array(con_arrs), axis=0)

# coherence gamma
inhibit_files = []
non_inhibit_files = []
for f_idx, f in enumerate(coh_gamma_files): 
    if lb_list_unique[f_idx] == 0:
        non_inhibit_files.append(f)
    if lb_list_unique[f_idx] == 1:
        inhibit_files.append(f)

con_arrs = []
for arr_file in inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_arrs.append(np.mean(con_arr, axis=0))
coh_inhibit_gamma_avg = np.mean(np.array(con_arrs), axis=0)
con_arrs = []
for arr_file in non_inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_arrs.append(np.mean(con_arr, axis=0))
coh_non_inhibit_gamma_avg = np.mean(np.array(con_arrs), axis=0)

# wpli alpha
inhibit_files = []
non_inhibit_files = []
for f_idx, f in enumerate(wpli_alpha_files): 
    if lb_list_unique[f_idx] == 0:
        non_inhibit_files.append(f)
    if lb_list_unique[f_idx] == 1:
        inhibit_files.append(f)

con_arrs = []
for arr_file in inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_arrs.append(np.mean(con_arr, axis=0))
wpli_inhibit_alpha_avg = np.mean(np.array(con_arrs), axis=0)
con_arrs = []
for arr_file in non_inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_arrs.append(np.mean(con_arr, axis=0))
wpli_non_inhibit_alpha_avg = np.mean(np.array(con_arrs), axis=0)

# wpli gamma
inhibit_files = []
non_inhibit_files = []
for f_idx, f in enumerate(wpli_gamma_files): 
    if lb_list_unique[f_idx] == 0:
        non_inhibit_files.append(f)
    if lb_list_unique[f_idx] == 1:
        inhibit_files.append(f)

con_arrs = []
for arr_file in inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_arrs.append(np.mean(con_arr, axis=0))
wpli_inhibit_gamma_avg = np.mean(np.array(con_arrs), axis=0)
con_arrs = []
for arr_file in non_inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_arrs.append(np.mean(con_arr, axis=0))
wpli_non_inhibit_gamma_avg = np.mean(np.array(con_arrs), axis=0)


In [ ]:
# statistical check of connectivity (first load and reshape data)
inhibit_files = []
non_inhibit_files = []
for f_idx, f in enumerate(coh_gamma_files): 
    if lb_list_unique[f_idx] == 0:
        non_inhibit_files.append(f)
    if lb_list_unique[f_idx] == 1:
        inhibit_files.append(f)

con_coh_inhibit_alpha = []
for arr_file in inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_coh_inhibit_alpha.append(np.mean(con_arr, axis=0))
coh_inhibit_alpha_avg = np.mean(np.array(con_coh_inhibit_alpha), axis=0)

con_coh_non_inhibit_alpha = []
for arr_file in non_inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_coh_non_inhibit_alpha.append(np.mean(con_arr, axis=0))
coh_non_inhibit_alpha_avg = np.mean(np.array(con_coh_non_inhibit_alpha), axis=0)

con_coh_inhibit_alpha_reshaped = np.transpose(np.array(con_coh_inhibit_alpha), axes=[1,2,0])
con_coh_non_inhibit_alpha_reshaped = np.transpose(np.array(con_coh_non_inhibit_alpha), axes=[1,2,0])

# second step check them in zip format

imp_brain_label_list = ['parahippocampal-lh', 'parahippocampal-rh', 'paracentral-lh', 'paracentral-rh',
                        'temporalpole-lh', 'temporalpole-rh', 'superiorfrontal-lh', 'superiorfrontal-rh',
                        'superiorparietal-lh', 'superiorparietal-rh', 'transversetemporal-lh', 'transversetemporal-rh',
                        'rostralanteriorcingulate-lh', 'rostralanteriorcingulate-rh', 'insula-lh', 'insula-rh',
                        'posteriorcingulate-lh', 'posteriorcingulate-rh', 'precentral-lh', 'precentral-rh',
                        'middletemporal-lh', 'middletemporal-rh', 'frontalpole-lh', 'frontalpole-rh',
                        'supramarginal-lh', 'supramarginal-rh'] 


p_thr = 0.1
for idx_i, idx_j in itertools.product(range(len(imp_brain_label_list)), range(len(imp_brain_label_list))):
    array_1 = con_coh_inhibit_alpha_reshaped[idx_i][idx_j]
    array_2 = con_coh_non_inhibit_alpha_reshaped[idx_i][idx_j]
    t_statistic, p_value = scipy.stats.ttest_ind(array_1, array_2, permutations=1000)
    if p_value < p_thr:
        print(f"significant difference between inhibition/non-inhibition within: {imp_brain_label_list[idx_i]} and {imp_brain_label_list[idx_j]} with p-value: {p_value}.")

In [ ]:
# plotting circles
node_names = ['parahippocampal-lh', 'transversetemporal-rh', 'pericalcarine-lh', 'temporalpole-rh',
                        'medialorbitofrontal-rh', 'lateraloccipital-rh', 'inferiortemporal-lh', 'superiortemporal-rh',
                        'supramarginal-rh', 'frontalpole-lh', 'parsopercularis-rh', 'posteriorcingulate-rh',
                        'superiorparietal-rh', 'parahippocampal-rh', 'rostralmiddlefrontal-lh',
                        'caudalmiddlefrontal-rh', 'parstriangularis-rh', 'lateraloccipital-lh', 'postcentral-lh'][:10]
# choose one of the following:
# coh_inhibit_alpha_avg, coh_non_inhibit_alpha_avg, coh_inhibit_gamma_avg, coh_non_inhibit_gamma_avg
# wpli_inhibit_alpha_avg, wpli_non_inhibit_alpha_avg, wpli_inhibit_gamma_avg, wpli_non_inhibit_gamma_avg
target = coh_inhibit_alpha_avg - coh_non_inhibit_alpha_avg

x = np.delete(target, 15, axis=0) # removing inferiortemporal-lh
target = np.delete(x, 15, axis=1)
#plot_connectivity_circle(target, node_names=node_names, n_lines=20,
#                        textcolor='black', node_edgecolor='white', colormap='viridis', fontsize_names=9,
#                        title=None, facecolor='white')

In [ ]:
reod_node_names = ['temporalpole-rh', 'superiortemporal-rh', 'supramarginal-rh', 'lateraloccipital-rh',
                    'caudalmiddlefrontal-rh', 'parsopercularis-rh', 'frontalpole-lh', 'rostralmiddlefrontal-lh',
                    'lateraloccipital-lh', 'pericalcarine-lh', 'postcentral-lh', 'inferiortemporal-lh', 
                    'parahippocampal-lh', 'parstriangularis-rh', 'transversetemporal-rh', 'superiorparietal-rh',
                    'posteriorcingulate-rh', 'parahippocampal-rh', 'medialorbitofrontal-rh'][:10] # added to separate alpha and gamma

new_array = np.zeros((10,10)) # 19 * 19
for new_id_1 in reod_node_names:
    for new_id_2 in reod_node_names:
        old_idx_1 = node_names.index(new_id_1)
        old_idx_2 = node_names.index(new_id_2)
        if old_idx_1 > old_idx_2:
            new_idx_1 = reod_node_names.index(new_id_1)
            new_idx_2 = reod_node_names.index(new_id_2)
            i = max(new_idx_1, new_idx_2)
            j = min(new_idx_1, new_idx_2)
            new_array[i][j] = target[old_idx_1][old_idx_2]

d_r = 180 / 13 # degree
d_l = 180 / 8
hd = d_r / 2 # half degree
node_angles = np.array([hd, 1*d_r+hd, 2*d_r+hd, 3*d_r+hd, 4*d_r+hd, 5*d_r+hd,
                        1*d_l+90, 2*d_l+90, 3*d_l+90, 4*d_l+90, 5*d_l+90, 6*d_l+90, 7*d_l+90,
                        -hd-5*d_r, -hd-4*d_r, -hd-3*d_r, -hd-2*d_r, -hd-1*d_r, -hd])

In [ ]:
plot_connectivity_circle(new_array, node_names=reod_node_names, n_lines=10, vmin=0.0, vmax=0.1,
                        textcolor='black', node_edgecolor='white', colormap='hot', fontsize_names=9, node_angles=node_angles,
                        title=None, facecolor='white')

In [ ]:
# new with symmetric lh/rh
node_names = ['parahippocampal-lh', 'parahippocampal-rh', 'transversetemporal-lh', 'transversetemporal-rh',
                        'pericalcarine-lh', 'pericalcarine-rh', 'temporalpole-lh', 'temporalpole-rh',
                        'medialorbitofrontal-lh', 'medialorbitofrontal-rh', 'lateraloccipital-lh', 'lateraloccipital-rh',
                        'inferiortemporal-lh', 'inferiortemporal-rh', 'superiortemporal-lh', 'superiortemporal-rh',
                        'supramarginal-lh', 'supramarginal-rh', 'frontalpole-lh', 'frontalpole-rh', 'parsopercularis-lh', 'parsopercularis-rh',
                        'posteriorcingulate-lh', 'posteriorcingulate-rh', 'superiorparietal-lh', 'superiorparietal-rh',
                        'rostralmiddlefrontal-lh', 'rostralmiddlefrontal-rh', 'inferiortemporal-lh', 'inferiortemporal-rh',
                        'caudalmiddlefrontal-lh', 'caudalmiddlefrontal-rh', 'parstriangularis-lh', 'parstriangularis-rh',
                        'lateraloccipital-lh', 'lateraloccipital-rh', 'postcentral-lh', 'postcentral-rh']

target = coh_inhibit_alpha_avg - coh_non_inhibit_alpha_avg


Feature explanation with SHAP

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test))

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer.shap_values(X_test)

In [ ]:
np.array(shap_values).shape

In [ ]:
# for connectivity
shap.summary_plot(shap_values[1], features=X_test, feature_names=None,
                    max_display=20, alpha=0.1, cmap='coolwarm') # barplots
# selected_features_idx[[96, 98, 99, 95, 94, 97, 92]] - 556

In [ ]:
# check the connection corresponding to this index
imp_brain_label_list = ['parahippocampal-lh', 'parahippocampal-rh', 'paracentral-lh', 'paracentral-rh',
                        'temporalpole-lh', 'temporalpole-rh', 'superiorfrontal-lh', 'superiorfrontal-rh',
                        'superiorparietal-lh', 'superiorparietal-rh', 'transversetemporal-lh', 'transversetemporal-rh',
                        'rostralanteriorcingulate-lh', 'rostralanteriorcingulate-rh', 'insula-lh', 'insula-rh',
                        'posteriorcingulate-lh', 'posteriorcingulate-rh', 'precentral-lh', 'precentral-rh',
                        'middletemporal-lh', 'middletemporal-rh', 'frontalpole-lh', 'frontalpole-rh',
                        'supramarginal-lh', 'supramarginal-rh'] 

imp_brain_label_list = ['vsn', 'smn', 'aun', 'van', 'dan', 'fpn', 'dmn', 'dgn', 'lbn']

texts = []
for index, imp in zip(np.array(keep_idxs)[selected_features_idx], np.sort(importance)[-n_sel_features:][:]):
    counter = 0
    if index - 36 >= 0:
        clas = 'gamma'
        new_index = index - 36
    else:
        clas = 'alpha'  
        new_index = index  
    for i, j in itertools.product(range(len(imp_brain_label_list)), range(len(imp_brain_label_list))):
        if j < i:
            counter += 1
            if counter == new_index:
                texts.append(f'{imp_brain_label_list[i]} and {imp_brain_label_list[j]} in {clas} with imp {imp}')

In [ ]:
texts

In [ ]:
np.save('/Users/payamsadeghishabestari/Prime/imp_connections_teas_300.npy', np.array(texts))

In [ ]:
# choosing the 10 most important indexes for each class of feature
d_p_idxs = []; t_p_idxs = []; a_p_idxs = []; b_p_idxs = []; g_p_idxs = []
d_e_idxs = []; t_e_idxs = []; a_e_idxs = []; b_e_idxs = []; g_e_idxs = []; ap_of_idxs = []; ap_sl_idxs = []
for idx, i in enumerate(np.array(keep_idxs)[selected_features_idx]):
    if 0*64 <= i < 1*64:
        d_p_idxs.append(idx)
    if 1*64 <= i < 2*64:
        t_p_idxs.append(idx)
    if 2*64 <= i < 3*64:
        a_p_idxs.append(idx)
    if 3*64 <= i < 4*64:
        b_p_idxs.append(idx)
    if 4*64 <= i < 5*64:
        g_p_idxs.append(idx)
    if 5*64 <= i < 6*64:
        d_e_idxs.append(idx)
    if 6*64 <= i < 7*64:
        t_e_idxs.append(idx)
    if 7*64 <= i < 8*64:
        a_e_idxs.append(idx)
    if 8*64 <= i < 9*64:
        b_e_idxs.append(idx)
    if 9*64 <= i < 10*64:
        g_e_idxs.append(idx)
    if 10*64 <= i < 11*64:
        ap_of_idxs.append(idx)
    if 11*64 <= i < 12*64:
        ap_sl_idxs.append(idx) 

In [ ]:
feature_sum_d_p = np.sum(X_test[:,d_p_idxs], axis=1)[:,np.newaxis]
feature_sum_t_p = np.sum(X_test[:,t_p_idxs], axis=1)[:,np.newaxis]
feature_sum_a_p = np.sum(X_test[:,a_p_idxs], axis=1)[:,np.newaxis]
feature_sum_b_p = np.sum(X_test[:,b_p_idxs], axis=1)[:,np.newaxis]
feature_sum_g_p = np.sum(X_test[:,g_p_idxs], axis=1)[:,np.newaxis]
feature_sum_d_e = np.sum(X_test[:,d_e_idxs], axis=1)[:,np.newaxis]
feature_sum_t_e = np.sum(X_test[:,t_e_idxs], axis=1)[:,np.newaxis]
feature_sum_a_e = np.sum(X_test[:,a_e_idxs], axis=1)[:,np.newaxis]
feature_sum_b_e = np.sum(X_test[:,b_e_idxs], axis=1)[:,np.newaxis]
feature_sum_g_e = np.sum(X_test[:,g_e_idxs], axis=1)[:,np.newaxis]
feature_sum_ap_of = np.sum(X_test[:,ap_of_idxs], axis=1)[:,np.newaxis]
feature_sum_ap_sl = np.sum(X_test[:,ap_sl_idxs], axis=1)[:,np.newaxis]
X_test_new = np.concatenate((feature_sum_d_p, feature_sum_t_p, feature_sum_a_p, feature_sum_b_p, feature_sum_g_p,
                            feature_sum_d_e, feature_sum_t_e, feature_sum_a_e, feature_sum_b_e, feature_sum_g_e,
                            feature_sum_ap_of, feature_sum_ap_sl),axis=1)

shap_d_p = np.sum(np.array(shap_values)[:,:,d_p_idxs], axis=2)[:,:,np.newaxis]
shap_t_p = np.sum(np.array(shap_values)[:,:,t_p_idxs], axis=2)[:,:,np.newaxis]
shap_a_p = np.sum(np.array(shap_values)[:,:,a_p_idxs], axis=2)[:,:,np.newaxis]
shap_b_p = np.sum(np.array(shap_values)[:,:,b_p_idxs], axis=2)[:,:,np.newaxis]
shap_g_p = np.sum(np.array(shap_values)[:,:,g_p_idxs], axis=2)[:,:,np.newaxis]
shap_d_e = np.sum(np.array(shap_values)[:,:,d_e_idxs], axis=2)[:,:,np.newaxis]
shap_t_e = np.sum(np.array(shap_values)[:,:,t_e_idxs], axis=2)[:,:,np.newaxis]
shap_a_e = np.sum(np.array(shap_values)[:,:,a_e_idxs], axis=2)[:,:,np.newaxis]
shap_b_e = np.sum(np.array(shap_values)[:,:,b_e_idxs], axis=2)[:,:,np.newaxis]
shap_g_e = np.sum(np.array(shap_values)[:,:,g_e_idxs], axis=2)[:,:,np.newaxis]
shap_ap_of = np.sum(np.array(shap_values)[:,:,ap_of_idxs], axis=2)[:,:,np.newaxis]
shap_ap_sl = np.sum(np.array(shap_values)[:,:,ap_sl_idxs], axis=2)[:,:,np.newaxis]

shap_new = np.concatenate((shap_d_p, shap_t_p, shap_a_p, shap_b_p, shap_g_p,
                            shap_d_e, shap_t_e, shap_a_e, shap_b_e, shap_g_e,
                            shap_ap_of, shap_ap_sl),axis=2)

feature_names = ['Delta power', 'Theta power', 'Alpha power', 'Beta power', 'Gamma power',
                    'Delta entropy', 'Theta entropy', 'Alpha entropy', 'Beta entropy', 'Gamma entropy',
                    'Aperiodic offset', 'Aperiodic Slope']

In [ ]:
shap_new = np.load('/Users/payamsadeghishabestari/Prime/shap/shap_new.npy',allow_pickle=True)
X_test_new = np.load('/Users/payamsadeghishabestari/Prime/shap/shap_features_rescaled_from_100_12.npy',allow_pickle=True)
feature_names = ['Delta power', 'Theta power', 'Alpha power', 'Beta power', 'Gamma power',
                    'Delta entropy', 'Theta entropy', 'Alpha entropy', 'Beta entropy', 'Gamma entropy',
                    'Aperiodic offset', 'Aperiodic Slope']

In [ ]:
shap.summary_plot(list(shap_new), features=X_test_new, feature_names=feature_names,
                    max_display=12, cmap='coolwarm', plot_size=(8,5)) # barplots

In [ ]:
shap.summary_plot(list(shap_new)[1], features=X_test_new, feature_names=feature_names,
                    max_display=12, alpha=0.6, cmap='coolwarm', plot_size=(8,5)) # barplots

In [ ]:
shap.dependence_plot("Alpha power", list(shap_new)[1], X_test_new, alpha=0.8,
                        feature_names=feature_names, interaction_index="Alpha entropy", cmap='coolwarm')

Checking Gamma power correlation (maybe)

In [ ]:
# RGB
fig, axs = plt.subplots(1,2)
epoch_ec = epochs_ec[0]
epoch_ec.info['bads'] = ['PO8', 'TP10', 'TP8', 'TP9', 'PO7', 'Fp2', 'Iz', 'FC6', 'CP5', 'TP7']
mne.viz.plot_sensors(epoch_ec.info, kind='topomap', pointsize=50, axes=axs[0], sphere=(0, 0.02, 0.01, 0.10))
epoch_ec.info['bads'] = ['PO8', 'AF4', 'AF8', 'C5', 'FC5', 'O2', 'TP9', 'F6', 'P8']
mne.viz.plot_sensors(epoch_ec.info, kind='topomap', pointsize=50, axes=axs[1], sphere=(0, 0.02, 0.01, 0.10))

In [ ]:
# TEAS
fig, axs = plt.subplots(1,2)
epoch_ec = epochs_ec[0]
epoch_ec.info['bads'] = ['CP3', 'CP5', 'P3', 'CP1', 'CP2', 'CPz', 'P2', 'P5', 'POz', 'P1']
mne.viz.plot_sensors(epoch_ec.info, kind='topomap', pointsize=50, axes=axs[0], sphere=(0, 0.02, 0.01, 0.10))
epoch_ec.info['bads'] = ['TP9', 'CP5', 'POz', 'AF7', 'T7', 'P5', 'FT7', 'TP8', 'F7', 'P7']
mne.viz.plot_sensors(epoch_ec.info, kind='topomap', pointsize=50, axes=axs[1], sphere=(0, 0.02, 0.01, 0.10))

In [ ]:
fs_dir = fetch_fsaverage(verbose=True)
src = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
sensors_colors = []
for sensor in epochs_ec[0].info['ch_names']:
    if sensor in ['PO8', 'TP10', 'TP8', 'TP9', 'PO7', 'Fp2', 'Iz', 'FC6', 'CP5', 'TP7']:
        sensors_colors.append('#ff7f0e')
    else:
        sensors_colors.append('#1f77b4')
fig = mne.viz.create_3d_figure(size=(800,800), bgcolor=(255, 255, 255))
fig= mne.viz.plot_alignment(
    epochs_ec[0].info, trans="fsaverage", subject="fsaverage", surfaces=dict(brain=0.2, outer_skull=0.2, head=0.1), coord_frame='head',
    eeg=["original"], meg=[], src=src, sensor_colors=sensors_colors,
    subjects_dir=None, fig=fig)

# Set viewing angle
mne.viz.set_3d_view(figure=fig, azimuth=135, elevation=80)

Connectivity plot with 20 labels

In [ ]:
node_names = ['parahippocampal-lh', 'parahippocampal-rh', 'paracentral-lh', 'paracentral-rh',
            'temporalpole-lh', 'temporalpole-rh', 'superiorfrontal-lh', 'superiorfrontal-rh',
            'superiorparietal-lh', 'superiorparietal-rh', 'transversetemporal-lh', 'transversetemporal-rh',
            'rostralanteriorcingulate-lh', 'rostralanteriorcingulate-rh', 'insula-lh', 'insula-rh',
            'posteriorcingulate-lh', 'posteriorcingulate-rh', 'precentral-lh', 'precentral-rh',
            'middletemporal-lh', 'middletemporal-rh', 'frontalpole-lh', 'frontalpole-rh',
            'supramarginal-lh', 'supramarginal-rh']

# choose one of the following:
# coh_inhibit_alpha_avg, coh_non_inhibit_alpha_avg, coh_inhibit_gamma_avg, coh_non_inhibit_gamma_avg
# wpli_inhibit_alpha_avg, wpli_non_inhibit_alpha_avg, wpli_inhibit_gamma_avg, wpli_non_inhibit_gamma_avg
target = coh_inhibit_gamma_avg - coh_non_inhibit_gamma_avg

# x = np.delete(target, obj=[10, 11, 12, 13], axis=0) # removing lateraloccipital and inferiortemporal
# target = np.delete(x, obj=[10, 11, 12, 13], axis=1)

reod_node_names = ['frontalpole-lh', 'frontalpole-rh', 'precentral-lh', 'precentral-rh',
                'superiorfrontal-lh', 'superiorfrontal-rh', 'rostralanteriorcingulate-lh', 'rostralanteriorcingulate-rh',
                'insula-lh', 'insula-rh', 'transversetemporal-lh', 'transversetemporal-rh',
                'middletemporal-lh', 'middletemporal-rh', 'temporalpole-lh', 'temporalpole-rh',
                'parahippocampal-lh', 'parahippocampal-rh', 'posteriorcingulate-lh', 'posteriorcingulate-rh',
                'superiorparietal-lh', 'superiorparietal-rh', 'supramarginal-lh', 'supramarginal-rh',
                'paracentral-lh', 'paracentral-rh'] 

new_array = np.zeros((len(reod_node_names),len(reod_node_names)))
for new_id_1 in reod_node_names:
    for new_id_2 in reod_node_names:
        old_idx_1 = node_names.index(new_id_1)
        old_idx_2 = node_names.index(new_id_2)
        if old_idx_1 > old_idx_2:
            new_idx_1 = reod_node_names.index(new_id_1)
            new_idx_2 = reod_node_names.index(new_id_2)
            i = max(new_idx_1, new_idx_2)
            j = min(new_idx_1, new_idx_2)
            new_array[i][j] = target[old_idx_1][old_idx_2]

angle_dist = 180 / ((len(reod_node_names) / 2) + 1)

node_angles = np.array([90+1*angle_dist, 90-1*angle_dist, 90+2*angle_dist, 90-2*angle_dist, 90+3*angle_dist, 90-3*angle_dist,
                        90+4*angle_dist, 90-4*angle_dist, 90+5*angle_dist, 90-5*angle_dist, 90+6*angle_dist, 90-6*angle_dist,
                        90+7*angle_dist, 90-7*angle_dist, 90+8*angle_dist, 90-8*angle_dist, 90+9*angle_dist, 90-9*angle_dist,
                        90+10*angle_dist, 90-10*angle_dist, 90+11*angle_dist, 90-11*angle_dist, 90+12*angle_dist, 90-12*angle_dist,
                        90+13*angle_dist, 90-13*angle_dist])

new_array_pos = np.maximum(new_array, 0)
new_array_neg = -np.minimum(new_array, 0)
fig, ax = plt.subplots(1,1, figsize=(11,11), subplot_kw={'projection': 'polar'})

# plot_connectivity_circle(new_array_neg, node_names=reod_node_names, n_lines=5, vmin=0.1, vmax=-0.1,
#                         textcolor='black', node_edgecolor='white', colormap='coolwarm', fontsize_names=13, node_angles=node_angles,
#                         title=None, facecolor='white', colorbar=False, linewidth=2, ax=ax)

plot_connectivity_circle(new_array_pos, node_names=reod_node_names, n_lines=5, vmin=-0.05, vmax=0.05,
                        textcolor='black', node_edgecolor='white', colormap='coolwarm',
                        fontsize_names=13, node_angles=node_angles, title=None, facecolor='white',
                        colorbar_pos=(0.9, 0.1), linewidth=2.5, ax=ax, node_width=10, node_height=1, node_colors=None)

In [ ]:
# grouping labels
node_names = ['parahippocampal-lh', 'parahippocampal-rh', 'paracentral-lh', 'paracentral-rh',
            'temporalpole-lh', 'temporalpole-rh', 'superiorfrontal-lh', 'superiorfrontal-rh',
            'superiorparietal-lh', 'superiorparietal-rh', 'transversetemporal-lh', 'transversetemporal-rh',
            'rostralanteriorcingulate-lh', 'rostralanteriorcingulate-rh', 'insula-lh', 'insula-rh',
            'posteriorcingulate-lh', 'posteriorcingulate-rh', 'precentral-lh', 'precentral-rh',
            'middletemporal-lh', 'middletemporal-rh', 'frontalpole-lh', 'frontalpole-rh',
            'supramarginal-lh', 'supramarginal-rh']

# choose one of the following:
# coh_inhibit_alpha_avg, coh_non_inhibit_alpha_avg, coh_inhibit_gamma_avg, coh_non_inhibit_gamma_avg
# wpli_inhibit_alpha_avg, wpli_non_inhibit_alpha_avg, wpli_inhibit_gamma_avg, wpli_non_inhibit_gamma_avg
target = coh_inhibit_gamma_avg - coh_non_inhibit_gamma_avg

# x = np.delete(target, obj=[10, 11, 12, 13], axis=0) # removing lateraloccipital and inferiortemporal
# target = np.delete(x, obj=[10, 11, 12, 13], axis=1)

reod_node_names = ['frontalpole-lh', 'frontalpole-rh', 'precentral-lh', 'precentral-rh', 'superiorfrontal-lh', 'superiorfrontal-rh',
                    'transversetemporal-lh', 'transversetemporal-rh', 'middletemporal-lh', 'middletemporal-rh', 'temporalpole-lh', 'temporalpole-rh',
                    'supramarginal-lh', 'supramarginal-rh', 'superiorparietal-lh', 'superiorparietal-rh', 'paracentral-lh', 'paracentral-rh',
                    'rostralanteriorcingulate-lh', 'rostralanteriorcingulate-rh', 'insula-lh', 'insula-rh',
                    'parahippocampal-lh', 'parahippocampal-rh', 'posteriorcingulate-lh', 'posteriorcingulate-rh'] 

new_array = np.zeros((len(reod_node_names),len(reod_node_names)))
for new_id_1 in reod_node_names:
    for new_id_2 in reod_node_names:
        old_idx_1 = node_names.index(new_id_1)
        old_idx_2 = node_names.index(new_id_2)
        if old_idx_1 > old_idx_2:
            new_idx_1 = reod_node_names.index(new_id_1)
            new_idx_2 = reod_node_names.index(new_id_2)
            i = max(new_idx_1, new_idx_2)
            j = min(new_idx_1, new_idx_2)
            new_array[i][j] = target[old_idx_1][old_idx_2]

angle_dist = 180 / 20

node_angles = np.array([90+1*angle_dist+10, 90-1*angle_dist-10, 90+2*angle_dist+10, 90-2*angle_dist-10, 90+3*angle_dist+10, 90-3*angle_dist-10,
                        90+6*angle_dist+5, 90-6*angle_dist-5, 90+7*angle_dist+5, 90-7*angle_dist-5, 90+8*angle_dist+5, 90-8*angle_dist-5,
                        90+11*angle_dist, 90-11*angle_dist, 90+12*angle_dist, 90-12*angle_dist, 90+13*angle_dist, 90-13*angle_dist,
                        90+16*angle_dist-10, 90-16*angle_dist+10, 90+17*angle_dist-10, 90-17*angle_dist+10, 90+18*angle_dist-10, 90-18*angle_dist+10,
                        90+19*angle_dist-10, 90-19*angle_dist+10])

cmap = matplotlib.cm.get_cmap('Set1')

node_colors_one_side = np.concatenate((np.array(cmap.colors[0])[:, np.newaxis], np.array(cmap.colors[0])[:, np.newaxis], np.array(cmap.colors[0])[:, np.newaxis],
                np.array(cmap.colors[1])[:, np.newaxis], np.array(cmap.colors[1])[:, np.newaxis], np.array(cmap.colors[1])[:, np.newaxis],
                np.array(cmap.colors[2])[:, np.newaxis], np.array(cmap.colors[2])[:, np.newaxis], np.array(cmap.colors[2])[:, np.newaxis],
                np.array(cmap.colors[3])[:, np.newaxis], np.array(cmap.colors[3])[:, np.newaxis],
                np.array(cmap.colors[3])[:, np.newaxis], np.array(cmap.colors[3])[:, np.newaxis],), axis=1)

node_colors = []
for line in node_colors_one_side.T:
    for _ in range(2):
        node_colors.append(line)
node_colors = np.array(node_colors)

new_array_pos = np.maximum(new_array, 0)
new_array_neg = -np.minimum(new_array, 0)
fig, ax = plt.subplots(1,1, figsize=(7,7), subplot_kw={'projection': 'polar'})

# plot_connectivity_circle(new_array_neg, node_names=reod_node_names, n_lines=5, vmin=0.1, vmax=-0.1,
#                         textcolor='black', node_edgecolor='white', colormap='coolwarm', fontsize_names=13, node_angles=node_angles,
#                         title=None, facecolor='white', colorbar=False, linewidth=2, ax=ax)

plot_connectivity_circle(new_array_pos, node_names=reod_node_names, n_lines=5, vmin=-0.05, vmax=0.05,
                        textcolor='black', node_edgecolor='white', colormap='coolwarm',
                        fontsize_names=10, node_angles=node_angles, title=None, facecolor='white',
                        colorbar_pos=(0.9, 0.1), linewidth=2.5, ax=ax, node_width=None, node_height=1, node_colors=node_colors)

Check label matching

In [ ]:
# match number between RGB and Teas (alpha)
teas_alpha_names = ['lateralorbitofrontal-lh', 'supramarginal-lh', 'lingual-lh', 'pericalcarine-rh', 'entorhinal-rh', 'transversetemporal-rh', 'transversetemporal-lh', 'precentral-rh', 'parsopercularis-lh', 'bankssts-lh', 'inferiorparietal-rh', 'superiorfrontal-lh', 'temporalpole-lh', 'parahippocampal-lh', 'rostralmiddlefrontal-lh', 'postcentral-lh', 'posteriorcingulate-rh', 'parstriangularis-lh', 'supramarginal-rh', 'superiorparietal-rh', 'superiortemporal-rh', 'lateraloccipital-lh', 'rostralanteriorcingulate-lh', 'caudalmiddlefrontal-lh']
RGB_alpha_names = ['temporalpole-rh', 'supramarginal-rh', 'inferiortemporal-lh', 'postcentral-lh', 'lingual-rh', 'lateraloccipital-rh', 'precentral-rh', 'parahippocampal-lh', 'rostralmiddlefrontal-lh', 'caudalmiddlefrontal-rh', 'posteriorcingulate-rh', 'parahippocampal-rh', 'parstriangularis-rh', 'lateraloccipital-lh', 'middletemporal-rh', 'superiorparietal-rh', 'parsopercularis-rh']
teas_gamma_names = ['inferiortemporal-rh', 'superiortemporal-rh', 'parahippocampal-rh', 'pericalcarine-rh', 'middletemporal-rh', 'transversetemporal-rh', 'superiorfrontal-rh', 'precentral-rh', 'entorhinal-rh', 'postcentral-rh', 'lingual-lh', 'bankssts-rh', 'transversetemporal-lh', 'parahippocampal-lh', 'rostralmiddlefrontal-lh', 'frontalpole-lh', 'middletemporal-lh', 'isthmuscingulate-rh', 'supramarginal-lh', 'lateraloccipital-lh', 'cuneus-lh', 'pericalcarine-lh', 'precentral-lh', 'lingual-rh', 'lateraloccipital-rh', 'insula-lh', 'rostralanteriorcingulate-lh', 'supramarginal-rh', 'superiortemporal-lh', 'cuneus-rh', 'superiorparietal-lh', 'caudalmiddlefrontal-lh', 'temporalpole-rh', 'inferiorparietal-rh', 'bankssts-lh', 'temporalpole-lh', 'parsopercularis-lh']
RGB_gamma_names = ['parsorbitalis-lh', 'rostralmiddlefrontal-lh', 'superiorfrontal-lh', 'inferiortemporal-rh', 'superiortemporal-lh', 'cuneus-rh', 'temporalpole-lh', 'posteriorcingulate-rh', 'inferiorparietal-rh', 'parahippocampal-rh', 'frontalpole-rh', 'superiorparietal-rh', 'parsorbitalis-rh', 'parstriangularis-lh', 'middletemporal-lh', 'supramarginal-lh', 'parsopercularis-rh', 'lateraloccipital-lh', 'rostralanteriorcingulate-rh', 'pericalcarine-lh', 'rostralmiddlefrontal-rh', 'superiortemporal-rh', 'parsopercularis-lh', 'medialorbitofrontal-rh', 'transversetemporal-lh', 'superiorfrontal-rh', 'lateraloccipital-rh', 'supramarginal-rh', 'superiorparietal-lh', 'transversetemporal-rh', 'pericalcarine-rh', 'frontalpole-lh', 'temporalpole-rh', 'inferiortemporal-lh', 'parahippocampal-lh']

counter_in = 0
counter_out = 0
for name in RGB_gamma_names[-10:]:
    if name in teas_gamma_names:
        counter_in += 1
    else:
        counter_out += 1
        print(name)


Check Correlation between high gamma power and other params (e.g HL)

In [ ]:
df = pd.read_excel('/Users/payamsadeghishabestari/Prime/PRIME_beh-data_complete.xlsx')

# sum gamma power for each epoch over 10 labels
features_array = np.load(file='/Users/payamsadeghishabestari/eeg_data/Regensburg/power_brain_features_rgb_mean.npy', allow_pickle=True)
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 69 labels
bl_idxs = range(len(brain_labels)) # 10
gammma_power_bl_idxs = 4 * 68 + np.array(bl_idxs)
summed_gamma = np.sum(features_array[:,gammma_power_bl_idxs], axis=1)

# which epoch is for which subject?
subject_ep_lens = []
for subj_file, subj_ep in zip(files_list_ec, epochs_ec): 
    subject_id = subj_file[-15:-11]
    subject_ep_lens.append(len(subj_ep))

def split_array(arr, lengths):
    if sum(lengths) != len(arr):
        raise ValueError("Sum of lengths must equal the length of the input array")
    split_arrays = []
    start = 0
    for length in lengths:
        end = start + length
        split_arrays.append(arr[start:end])
        start = end
    return split_arrays

result = split_array(summed_gamma, subject_ep_lens)
gamma_powers = [np.mean(i) for i in result]
df['gamma_powers'] = gamma_powers

# choose only gamma values larger than threshold
gamma_thr = np.median(gamma_powers) 
sel_df = df[df['gamma_powers'] > gamma_thr]
print(len(sel_df))

new_df_1 = sel_df[sel_df['study']=='AAEEG']
new_df_1['Binary suppresion'] = (new_df_1['WN_TL_0'] <= 90).astype(int)
new_df_2 = sel_df[sel_df['study']=='neuropren']
new_df_2['Binary suppresion'] = (new_df_2['b1_WN_0'] <= 90).astype(int)
new_sel_df = pd.concat([new_df_1, new_df_2], axis=0)

correlation = new_sel_df['HA'].corr(new_sel_df['Binary suppresion'])

In [ ]:
p_values = {}
chi_values = {}
for item in ['age', 'sex', 'THI', 'HL', 'tin_duration', 'tin_frequency', 'MML', 'tin_loud', 'HA']:
    contingency_table = pd.crosstab(new_sel_df[item], new_sel_df['Binary suppresion'])
    chi2, p, _, _ = chi2_contingency(contingency_table, correction=False) # or True
    p_values[item] = p
    chi_values[item] = chi2


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3,4))
sns.violinplot(data=new_sel_df, y='HA', x='Binary suppresion', bw=.3, saturation=0.3, palette='Set2', inner=None, ax=ax)
sns.swarmplot(data=new_sel_df, y='HA', x='Binary suppresion', dodge=False, size=4, palette='Set2', ax=ax)
ax.set_ylim([-10, 30])

Jaccard Similarity

In [ ]:
counter_in = 0
counter_out = 0
for name in rgb_list_alpha[:10]:
    if name[:-2] in [t[:-2] for t in tinnoice_list_alpha]:
        counter_in += 1
    else:
        counter_out += 1
        # print(name)
print(f'number of matches:{counter_in}')
print(f'number of mismatches:{counter_out}')


Check connectivity in teas and tinnoice

In [ ]:
# loading labeling
coh_alpha_files = []; coh_gamma_files = []
lb_list_unique = []
for label in label_list:
    lb_list_unique.append(label[0])

# loading and sorting files
folder_path = '/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/both_hemi/Tinnoice'  
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    files_in_folder = sorted(os.listdir(folder_path), key=lambda x: os.path.getmtime(os.path.join(folder_path, x)))
    for filename in files_in_folder:
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and file_path.endswith(".npy"):
            if 'coh_alpha' in file_path:
                coh_alpha_files.append(file_path)
            if 'coh_gamma' in file_path:
                coh_gamma_files.append(file_path)

# coherence alpha
inhibit_files = []
non_inhibit_files = []
for f_idx, f in enumerate(coh_alpha_files): 
    if lb_list_unique[f_idx] == 0:
        non_inhibit_files.append(f)
    if lb_list_unique[f_idx] == 1:
        inhibit_files.append(f)

con_arrs = []
for arr_file in inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_arrs.append(np.mean(con_arr, axis=0))
coh_inhibit_alpha_avg = np.mean(np.array(con_arrs), axis=0)
con_arrs = []
for arr_file in non_inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_arrs.append(np.mean(con_arr, axis=0))
coh_non_inhibit_alpha_avg = np.mean(np.array(con_arrs), axis=0)

# coherence gamma
inhibit_files = []
non_inhibit_files = []
for f_idx, f in enumerate(coh_gamma_files): 
    if lb_list_unique[f_idx] == 0:
        non_inhibit_files.append(f)
    if lb_list_unique[f_idx] == 1:
        inhibit_files.append(f)

con_arrs = []
for arr_file in inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_arrs.append(np.mean(con_arr, axis=0))
coh_inhibit_gamma_avg = np.mean(np.array(con_arrs), axis=0)
con_arrs = []
for arr_file in non_inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_arrs.append(np.mean(con_arr, axis=0))
coh_non_inhibit_gamma_avg = np.mean(np.array(con_arrs), axis=0)

# statistical check of connectivity (first load and reshape data)
inhibit_files = []
non_inhibit_files = []
for f_idx, f in enumerate(coh_alpha_files): 
    if lb_list_unique[f_idx] == 0:
        non_inhibit_files.append(f)
    if lb_list_unique[f_idx] == 1:
        inhibit_files.append(f)

con_coh_inhibit_alpha = []
for arr_file in inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_coh_inhibit_alpha.append(np.mean(con_arr, axis=0))
coh_inhibit_alpha_avg = np.mean(np.array(con_coh_inhibit_alpha), axis=0)

con_coh_non_inhibit_alpha = []
for arr_file in non_inhibit_files:
    con_arr = np.load(arr_file, allow_pickle=True)
    con_coh_non_inhibit_alpha.append(np.mean(con_arr, axis=0))
coh_non_inhibit_alpha_avg = np.mean(np.array(con_coh_non_inhibit_alpha), axis=0)

In [ ]:
# TEAS

con_coh_inhibit_alpha_reshaped = np.transpose(np.array(con_coh_inhibit_alpha), axes=[1,2,0])
con_coh_non_inhibit_alpha_reshaped = np.transpose(np.array(con_coh_non_inhibit_alpha), axes=[1,2,0])

# second step check them in zip format

brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 69 labels
node_names = [lb.name for lb in brain_labels]
p_thr = 0.02
for idx_i, idx_j in itertools.product(range(68), range(68)):
    array_1 = con_coh_inhibit_alpha_reshaped[idx_i][idx_j]
    array_2 = con_coh_non_inhibit_alpha_reshaped[idx_i][idx_j]
    t_statistic, p_value = scipy.stats.ttest_ind(array_1, array_2, permutations=1000)
    if p_value < p_thr:
        print(f"significant difference between inhibition/non-inhibition within: {brain_labels[idx_i].name} and {brain_labels[idx_j].name} with p-value: {p_value}.")

In [ ]:
new_array = coh_non_inhibit_alpha_avg - coh_inhibit_alpha_avg
new_array_pos = np.maximum(new_array, 0)
new_array_neg = -np.minimum(new_array, 0)
fig, ax = plt.subplots(1,1, figsize=(11,11), subplot_kw={'projection': 'polar'})

# plot_connectivity_circle(new_array_neg, node_names=node_names, n_lines=10, vmin=0.1, vmax=-0.1,
#                         textcolor='black', node_edgecolor='white', colormap='coolwarm', fontsize_names=9, node_angles=None,
#                         title=None, facecolor='white', colorbar=False, ax=ax)

plot_connectivity_circle(new_array_pos, node_names=node_names, n_lines=10, vmin=-0.1, vmax=0.1,
                        textcolor='black', node_edgecolor='white', colormap='coolwarm', fontsize_names=9, node_angles=None,
                        title=None, facecolor='white', colorbar_pos=(0.9, 0.1), ax=ax)

making connectivity plots with important connections

In [ ]:
x = np.load('/Users/payamsadeghishabestari/Prime/imp_connections_rgb.npy', allow_pickle=True)
for item in x:
    if 'alpha' in item:
        print(item)

In [ ]:
reod_node_names = ['frontalpole-lh', 'frontalpole-rh', 'precentral-lh', 'precentral-rh', 'superiorfrontal-lh', 'superiorfrontal-rh',
                    'transversetemporal-lh', 'transversetemporal-rh', 'middletemporal-lh', 'middletemporal-rh', 'temporalpole-lh', 'temporalpole-rh',
                    'supramarginal-lh', 'supramarginal-rh', 'superiorparietal-lh', 'superiorparietal-rh', 'paracentral-lh', 'paracentral-rh',
                    'rostralanteriorcingulate-lh', 'rostralanteriorcingulate-rh', 'insula-lh', 'insula-rh',
                    'parahippocampal-lh', 'parahippocampal-rh', 'posteriorcingulate-lh', 'posteriorcingulate-rh'] 

new_array = np.zeros((26,26))

# alpha
# new_array = np.zeros((26,26))
# new_array[25][12]= 0.004180300120799728
# new_array[25][21]= 0.004051962505621442
# new_array[13][10]= 0.003706030012896151
# new_array[23][2]= 0.0033120632978689253
# new_array[10][9]= 0.003056123686813693
# new_array[7][1]= 0.0027807497494880978
# new_array[19][4]= 0.0027416196706699213
# new_array[23][14]= 0.002682994188305871
# new_array[5][2]= 0.0024406396970428875
# new_array[22][11]= 0.0024049593114350656

# gamma
# new_array = np.zeros((26,26))
# new_array[23][2]= 0.008337426164523739
# new_array[17][16]= 0.006929944791816353
# new_array[25][21]= 0.005707022689821889
# new_array[19][2]= 0.005145763659568517
# new_array[19][4]= 0.0046119613185669
# new_array[20][7]= 0.0044442687154322615
# new_array[20][13]= 0.004399014207988937
# new_array[15][5]= 0.004312046073316636
# new_array[5][2]= 0.004205914372212887
# new_array[16][15]= 0.004198167838068725



new_array *= 100

angle_dist = 180 / 20

node_angles = np.array([90+1*angle_dist+10, 90-1*angle_dist-10, 90+2*angle_dist+10, 90-2*angle_dist-10, 90+3*angle_dist+10, 90-3*angle_dist-10,
                        90+6*angle_dist+5, 90-6*angle_dist-5, 90+7*angle_dist+5, 90-7*angle_dist-5, 90+8*angle_dist+5, 90-8*angle_dist-5,
                        90+11*angle_dist, 90-11*angle_dist, 90+12*angle_dist, 90-12*angle_dist, 90+13*angle_dist, 90-13*angle_dist,
                        90+16*angle_dist-10, 90-16*angle_dist+10, 90+17*angle_dist-10, 90-17*angle_dist+10, 90+18*angle_dist-10, 90-18*angle_dist+10,
                        90+19*angle_dist-10, 90-19*angle_dist+10])

cmap = matplotlib.cm.get_cmap('Set1')

node_colors_one_side = np.concatenate((np.array(cmap.colors[0])[:, np.newaxis], np.array(cmap.colors[0])[:, np.newaxis], np.array(cmap.colors[0])[:, np.newaxis],
                np.array(cmap.colors[1])[:, np.newaxis], np.array(cmap.colors[1])[:, np.newaxis], np.array(cmap.colors[1])[:, np.newaxis],
                np.array(cmap.colors[2])[:, np.newaxis], np.array(cmap.colors[2])[:, np.newaxis], np.array(cmap.colors[2])[:, np.newaxis],
                np.array(cmap.colors[3])[:, np.newaxis], np.array(cmap.colors[3])[:, np.newaxis],
                np.array(cmap.colors[3])[:, np.newaxis], np.array(cmap.colors[3])[:, np.newaxis],), axis=1)

node_colors = []
for line in node_colors_one_side.T:
    for _ in range(2):
        node_colors.append(line)
node_colors = np.array(node_colors)

new_array_pos = np.maximum(new_array, 0)
new_array_neg = -np.minimum(new_array, 0)
fig, ax = plt.subplots(1,1, figsize=(7,7), subplot_kw={'projection': 'polar'})

# plot_connectivity_circle(new_array_neg, node_names=reod_node_names, n_lines=5, vmin=0.1, vmax=-0.1,
#                         textcolor='black', node_edgecolor='white', colormap='coolwarm', fontsize_names=13, node_angles=node_angles,
#                         title=None, facecolor='white', colorbar=False, linewidth=2, ax=ax)

plot_connectivity_circle(new_array_pos, node_names=reod_node_names, n_lines=10, vmin=0, vmax=0.5,
                        textcolor='black', node_edgecolor='white', colormap='coolwarm',
                        fontsize_names=10, node_angles=node_angles, title=None, facecolor='white',
                        colorbar_pos=(0.9, 0.1), linewidth=2.5, ax=ax, node_width=None, node_height=1, node_colors=node_colors)

In [ ]:
texts = np.load('/Users/payamsadeghishabestari/Prime/imp_connections_rgb.npy', allow_pickle=True)
g_text = [text for text in texts if 'gamma' in text]
g_text[-10:]

In [ ]:
# gamma
new_array = np.zeros((26,26))
new_array[23][2]= 0.008337426164523739
new_array[17][16]= 0.006929944791816353
new_array[25][21]= 0.005707022689821889
new_array[19][2]= 0.005145763659568517
new_array[19][4]= 0.0046119613185669
new_array[20][7]= 0.0044442687154322615
new_array[20][13]= 0.004399014207988937

labeling with alpha and gamma (brain labels)

In [ ]:
# plotting brain with colored labels
# initiating the brain
Brain = mne.viz.get_brain_class()
brain_kwargs = dict(alpha=0.15, background="white", cortex="low_contrast", size=(800, 600), views='medial')
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc') # 69 labels
imp_brain_label_list_gamma = ['parahippocampal-lh', 'transversetemporal-rh', 'pericalcarine-lh', 'temporalpole-rh',
                        'medialorbitofrontal-rh', 'lateraloccipital-rh', 'inferiortemporal-lh', 'superiortemporal-rh',
                        'supramarginal-rh', 'frontalpole-lh']
imp_brain_label_list_alpha = ['parsopercularis-rh', 'posteriorcingulate-rh', 'superiorparietal-rh', 'parahippocampal-rh',
                            'rostralmiddlefrontal-lh', 'inferiortemporal-lh', 'caudalmiddlefrontal-rh', 'parstriangularis-rh',
                            'lateraloccipital-lh', 'postcentral-lh']
imp_lb_a_rh_idxs = []; imp_lb_a_lh_idxs = []
imp_lb_g_rh_idxs = []; imp_lb_g_lh_idxs = []

# indexing right/left and alpha/gamma labels
for im_l_a, im_l_g in zip(imp_brain_label_list_alpha, imp_brain_label_list_gamma):
    for l_idx, l_id in enumerate(brain_labels):
        if l_id.name == im_l_a and l_id.name[-2:] == 'rh':
            imp_lb_a_rh_idxs.append(l_idx)
        if l_id.name == im_l_a and l_id.name[-2:] == 'lh':
            imp_lb_a_lh_idxs.append(l_idx)
        if l_id.name == im_l_g and l_id.name[-2:] == 'rh':
            imp_lb_g_rh_idxs.append(l_idx)
        if l_id.name == im_l_g and l_id.name[-2:] == 'lh':
            imp_lb_g_lh_idxs.append(l_idx)

# plotting
brain = Brain("fsaverage", hemi="rh", surf="pial", title='alpha-rh', **brain_kwargs)
for label in list(np.array(brain_labels)[imp_lb_a_rh_idxs]): # alpha-rh
    brain.add_label(label, hemi="rh", color="#1f77b4", borders=False, alpha=0.5)
# brain.save_image(filename='/Users/payamsadeghishabestari/Prime/parcel-medial-alpha-rh.png', mode='rgb')

brain = Brain("fsaverage", hemi="lh", surf="pial", title='alpha-lh', **brain_kwargs)
for label in list(np.array(brain_labels)[imp_lb_a_lh_idxs]): # alpha-lh
    brain.add_label(label, hemi="lh", color="#1f77b4", borders=False, alpha=0.5)
# brain.save_image(filename='/Users/payamsadeghishabestari/Prime/parcel-medial-alpha-lh.png', mode='rgb')


brain = Brain("fsaverage", hemi="rh", surf="pial", title='gamma-rh', **brain_kwargs)
for label in list(np.array(brain_labels)[imp_lb_g_rh_idxs]): # gamma-rh
    brain.add_label(label, hemi="rh", color="#ff7f0e", borders=False, alpha=0.5)
# brain.save_image(filename='/Users/payamsadeghishabestari/Prime/parcel-medial-gamma-rh.png', mode='rgb')


brain = Brain("fsaverage", hemi="lh", surf="pial", title='gamma-lh', **brain_kwargs)
for label in list(np.array(brain_labels)[imp_lb_g_lh_idxs]): # gamma-lh
    brain.add_label(label, hemi="lh", color="#ff7f0e", borders=False, alpha=0.5)
# brain.save_image(filename='/Users/payamsadeghishabestari/Prime/parcel-medial-gamma-lh.png', mode='rgb')

In [ ]:
Brain = mne.viz.get_brain_class()
clr = 0.85
brain_kwargs = dict(alpha=1, background="white", cortex=[(clr, clr, clr), (clr, clr, clr)], size=(800, 600), views='lateral')
silhouette_dict = {'color':(0.8, 0.8, 0.8), 'linewidth':1, 'alpha':0.8, 'decimate':0.9}
brain = Brain("fsaverage", hemi="rh", surf="pial_semi_inflated", silhouette=silhouette_dict, **brain_kwargs)
# brain.save_image(filename='/Users/payamsadeghishabestari/Prime/parcel-lateral-rh.png', mode='rgb')

Compare TEAS and RGB

In [ ]:
imp_con_rgb = np.load('/Users/payamsadeghishabestari/Prime/imp_connections_rgb.npy', allow_pickle=True)
imp_con_teas = np.load('/Users/payamsadeghishabestari/Prime/imp_connections_teas_300.npy', allow_pickle=True)

In [ ]:
for element in imp_con_teas:
    if 'superiorparietal' in element and 'paracentral' in element and 'gamma' in element:
        print(element)

Checking and plotting Aperiodic component

In [ ]:
ep = 2
ep_idx = 0
psds, freqs = epochs_ec[ep][ep_idx].compute_psd(method='multitaper', picks='eeg', fmin=0.5, fmax=80, bandwidth=6, verbose=False).get_data(return_freqs=True)
fm = FOOOF(max_n_peaks=5)
spectrum = np.squeeze(psds)[19] # channel Iz
fm.fit(freqs, spectrum, freq_range=[0.5, 80])
fm.report(freqs, spectrum, [0.5, 80])

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,5))
plot_kwargs = {'color' : 'k', 'alpha' : 0.7, 'lw' : 1.5}
fooof.plts.fm.plot_fm(fm, plot_peaks='shade', ax=ax)
ax.grid(visible=False, which='minor', axis='y')
ax.legend(fontsize=10, frameon=False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

In [ ]:
ap_comp = 10**(-9.5404) * 1/(0 + freqs**0.8647)

delta_fill = freqs[:9]
theta_fill = freqs[8:17]
alpha_fill = freqs[16:27]
beta_fill = freqs[30:60]
gamma_fill = freqs[59:]

fig, ax = plt.subplots(1,1,figsize=(8,5))



ax.plot(freqs, spectrum, linewidth=0.5, color='grey')
ax.plot(freqs, ap_comp, linewidth=2.5, linestyle='--')
ax.plot(freqs, 10 ** fm.fooofed_spectrum_, linewidth=2)

ax.fill_between(x=freqs[1:10], y1=ap_comp[1:10], y2=10 ** fm.fooofed_spectrum_[1:10], alpha=0.2, color='grey')
ax.fill_between(x=freqs[6:28], y1=ap_comp[6:28], y2=10 ** fm.fooofed_spectrum_[6:28], alpha=0.2, color='grey')
ax.fill_between(x=freqs[73:110], y1=ap_comp[73:110], y2=10 ** fm.fooofed_spectrum_[73:110], alpha=0.2, color='grey')
ax.fill_between(x=freqs[115:134], y1=ap_comp[115:134], y2=10 ** fm.fooofed_spectrum_[115:134], alpha=0.2, color='grey')
ax.fill_between(x=freqs[114:159], y1=ap_comp[114:159], y2=10 ** fm.fooofed_spectrum_[114:159], alpha=0.2, color='grey')

ax.fill_between(delta_fill, y1=ap_comp[:9], y2=0, alpha=0.5, color='#d62728', label='delta')
ax.fill_between(theta_fill, y1=ap_comp[8:17], y2=0, alpha=0.5, color='#2ca02c', label='theta') # theta
ax.fill_between(alpha_fill, y1=ap_comp[16:27], y2=0, alpha=0.5, color='#9467bd', label='alpha') # alpha
ax.fill_between(beta_fill, y1=ap_comp[30:60], y2=0, alpha=0.5, color='#bcbd22', label='beta') # beta
ax.fill_between(gamma_fill, y1=ap_comp[59:], y2=0, alpha=0.5, color='#17becf', label='gamma') # gamma
ax.set_yscale('log')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(frameon=False)
ax.set_ylim([0, 10**-9])

Connectivity between networks

In [ ]:
# making equal channel size
for ep in epochs_ec:
    ep.drop_channels(['HRli', 'HRre'], on_missing='warn')

fs_dir = fetch_fsaverage(verbose=False)
trans = "fsaverage"  
src = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
bem = op.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")
method = "dSPM"
snr = 3.0
lambda2 = 1.0 / snr**2
sfreq = 250
counter = 0
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 69 labels
bl_names = [bl.name for bl in brain_labels]

In [ ]:
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 69 labels
bl_names = [bl.name for bl in brain_labels]

vsn_lbs = ['cuneus-lh', 'cuneus-rh',
'lingual-lh', 'lingual-rh',
'lateraloccipital-lh', 'lateraloccipital-rh',
'pericalcarine-lh', 'pericalcarine-rh'] 

smn_lbs = ['caudalmiddlefrontal-lh', 'caudalmiddlefrontal-rh',
'postcentral-lh', 'postcentral-rh',
'precentral-lh', 'precentral-rh',
'paracentral-lh', 'paracentral-rh',
'transversetemporal-lh', 'transversetemporal-rh']

aun_lbs = ['superiorparietal-lh', 'superiorparietal-rh',
'superiortemporal-lh', 'superiortemporal-rh',
'transversetemporal-lh', 'transversetemporal-rh']

van_lbs = ['fusiform-lh', 'fusiform-rh',
'inferiorparietal-lh', 'inferiorparietal-rh',
'lingual-lh', 'lingual-rh',
'lateraloccipital-lh', 'lateraloccipital-rh']

dan_lbs = ['caudalmiddlefrontal-lh', 'caudalmiddlefrontal-rh',
'lateraloccipital-lh', 'lateraloccipital-rh',
'paracentral-lh', 'paracentral-rh',
'superiorparietal-lh', 'superiorparietal-rh',
'superiortemporal-lh', 'superiortemporal-rh']

fpn_lbs = ['lateralorbitofrontal-lh', 'lateralorbitofrontal-rh',
'parsopercularis-lh', 'parsopercularis-rh',
'parsorbitalis-lh', 'parsorbitalis-rh',
'parstriangularis-lh', 'parstriangularis-rh',
'rostralmiddlefrontal-lh', 'rostralmiddlefrontal-rh',
'superiorfrontal-lh', 'superiorfrontal-rh',
'superiortemporal-lh', 'superiortemporal-rh']

dmn_lbs = ['caudalanteriorcingulate-lh', 'caudalanteriorcingulate-rh',
'entorhinal-lh', 'entorhinal-rh',
'frontalpole-lh', 'frontalpole-rh',
'isthmuscingulate-lh', 'isthmuscingulate-rh',
'medialorbitofrontal-lh', 'medialorbitofrontal-rh',
'parahippocampal-lh', 'parahippocampal-rh',
'posteriorcingulate-lh', 'posteriorcingulate-rh',
'precuneus-lh', 'precuneus-rh',
'rostralanteriorcingulate-lh', 'rostralanteriorcingulate-rh']

dgn_lbs = ['caudalmiddlefrontal-lh', 'caudalmiddlefrontal-rh',
'paracentral-lh', 'paracentral-rh',
'postcentral-lh', 'postcentral-rh',
'precentral-lh', 'precentral-rh',
'superiorparietal-lh', 'superiorparietal-rh',
'superiortemporal-lh', 'superiortemporal-rh']

lbn_lbs = ['caudalanteriorcingulate-lh', 'caudalanteriorcingulate-rh',
'entorhinal-lh', 'entorhinal-rh',
'frontalpole-lh', 'frontalpole-rh',
'isthmuscingulate-lh', 'isthmuscingulate-rh',
'medialorbitofrontal-lh', 'medialorbitofrontal-rh',
'parahippocampal-lh', 'parahippocampal-rh',
'posteriorcingulate-lh', 'posteriorcingulate-rh']

In [ ]:
index = bl_names.index(vsn_lbs[0])
vsn_n = brain_labels[index]
for bl in vsn_lbs[1:]:
    index = bl_names.index(bl)
    vsn_n += brain_labels[index]

index = bl_names.index(smn_lbs[0])
smn_n = brain_labels[index]
for bl in smn_lbs[1:]:
    index = bl_names.index(bl)
    smn_n += brain_labels[index]

index = bl_names.index(aun_lbs[0])
aun_n = brain_labels[index]
for bl in aun_lbs[1:]:
    index = bl_names.index(bl)
    aun_n += brain_labels[index]

index = bl_names.index(van_lbs[0])
van_n = brain_labels[index]
for bl in van_lbs[1:]:
    index = bl_names.index(bl)
    van_n += brain_labels[index]

index = bl_names.index(dan_lbs[0])
dan_n = brain_labels[index]
for bl in dan_lbs[1:]:
    index = bl_names.index(bl)
    dan_n += brain_labels[index]

index = bl_names.index(fpn_lbs[0])
fpn_n = brain_labels[index]
for bl in fpn_lbs[1:]:
    index = bl_names.index(bl)
    fpn_n += brain_labels[index]

index = bl_names.index(dmn_lbs[0])
dmn_n = brain_labels[index]
for bl in dmn_lbs[1:]:
    index = bl_names.index(bl)
    dmn_n += brain_labels[index]

index = bl_names.index(dgn_lbs[0])
dgn_n = brain_labels[index]
for bl in dgn_lbs[1:]:
    index = bl_names.index(bl)
    dgn_n += brain_labels[index]

index = bl_names.index(lbn_lbs[0])
lbn_n = brain_labels[index]
for bl in lbn_lbs[1:]:
    index = bl_names.index(bl)
    lbn_n += brain_labels[index]

sel_brain_labels = []
sel_brain_labels.append(vsn_n)
sel_brain_labels.append(smn_n)
sel_brain_labels.append(aun_n)
sel_brain_labels.append(van_n)
sel_brain_labels.append(dan_n)
sel_brain_labels.append(fpn_n)
sel_brain_labels.append(dmn_n)
sel_brain_labels.append(dgn_n)
sel_brain_labels.append(lbn_n)

In [ ]:
# looping over subjects to compute connectivity
for ep in tqdm(epochs_ec[:]):
    # initiating
    ep.set_eeg_reference('average', projection=True)
    ep.apply_proj()
    fwd = mne.make_forward_solution(ep.info, trans=trans, src=src, bem=bem, eeg=True, mindist=5.0, verbose=False)
    noise_cov = mne.make_ad_hoc_cov(ep.info, std=None, verbose=False)
    inverse_operator = make_inverse_operator(ep.info, fwd, noise_cov, loose=0.2, depth=0.8, verbose=False)
    # computing source estimate over all surface
    stc = apply_inverse_epochs(ep, inverse_operator, lambda2=lambda2,
                                                method=method, label=None,
                                                return_generator=False, verbose=False)
    # project it from whole surface to labels
    label_ts = mne.extract_label_time_course(stc, sel_brain_labels, inverse_operator['src'], mode='mean',
                                            allow_empty=True, return_generator=False, verbose=False)

    freqs = np.linspace(8, 13, 5)
    con = spectral_connectivity_time(np.array(label_ts), freqs=freqs, method='coh', average=False, sfreq=sfreq,
                                    fmin=freqs[0], fmax=freqs[-1], mode='cwt_morlet', n_cycles=7,
                                    faverage=True, n_jobs=1, verbose=False)
    conmat_coh = con.get_data(output='dense')[:, :, :, 0]
    fname_coh = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/networks_connectivity/con_coh_alpha_{counter}.npy'
    np.save(fname_coh, conmat_coh)
    
    freqs = np.linspace(30, 80, 5)
    con = spectral_connectivity_time(np.array(label_ts), freqs=freqs, method='coh', average=False, sfreq=sfreq,
                                    fmin=freqs[0], fmax=freqs[-1], mode='cwt_morlet', n_cycles=7,
                                    faverage=True, n_jobs=1, verbose=False)
    conmat_coh = con.get_data(output='dense')[:, :, :, 0]
    fname_coh = f'/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/networks_connectivity/con_coh_gamma_{counter}.npy'
    np.save(fname_coh, conmat_coh)
    
    counter += 1

In [ ]:
# check the connection corresponding to this index
imp_brain_label_list = ['parahippocampal-lh', 'parahippocampal-rh', 'paracentral-lh', 'paracentral-rh',
                        'temporalpole-lh', 'temporalpole-rh', 'superiorfrontal-lh', 'superiorfrontal-rh',
                        'superiorparietal-lh', 'superiorparietal-rh', 'transversetemporal-lh', 'transversetemporal-rh',
                        'rostralanteriorcingulate-lh', 'rostralanteriorcingulate-rh', 'insula-lh', 'insula-rh',
                        'posteriorcingulate-lh', 'posteriorcingulate-rh', 'precentral-lh', 'precentral-rh',
                        'middletemporal-lh', 'middletemporal-rh', 'frontalpole-lh', 'frontalpole-rh',
                        'supramarginal-lh', 'supramarginal-rh'] 

imp_brain_label_list = ['vsn', 'smn', 'aun', 'van', 'dan', 'fpn', 'dmn', 'dgn', 'lbn']

texts = []
for index, imp in zip(np.array(keep_idxs)[selected_features_idx], np.sort(importance)[-n_sel_features:][:]):
    counter = 0
    if index >= 36:
        clas = 'gamma'
        new_index = index - 36
    
    if index < 36:
        clas = 'alpha'
        new_index = index
    

    for i, j in itertools.product(range(len(imp_brain_label_list)), range(len(imp_brain_label_list))):
        if j < i:
            # counter += 1
            if counter == new_index and clas=='alpha':
                texts.append(f'{imp_brain_label_list[i]} and {imp_brain_label_list[j]} in {clas} with imp {imp}')
            counter += 1


Interconnections between networks + Auditory intra network

In [ ]:
model = RandomForestClassifier(n_estimators=100)
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=None)
mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1

n_scores = cross_val_score(model, X_train, y_train, groups = group_train ,scoring='accuracy', cv=10, n_jobs=-1)
model.fit(X_train, y_train)

# reporting and features overview
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer.shap_values(X_test)

In [ ]:
networks = ['vsn', 'smn', 'aun', 'van', 'dan', 'fpn', 'dmn', 'dgn', 'lbn']
aud_network = ['superiorparietal-lh', 'superiorparietal-rh',
                'superiortemporal-lh', 'superiortemporal-rh',
                'transversetemporal-lh', 'transversetemporal-rh']

feature_names = []
for k_idx in np.array(keep_idxs):
    counter = 0
    if k_idx < 36: # corresponds to inter networks (alpha)
        for i, j in product(networks, networks):
            if networks.index(i) < networks.index(j):
                if k_idx == counter:
                    feature_names.append(f'{i} & {j} in alpha')
                counter += 1
    
    if 36 <= k_idx < 72: # corresponds to inter networks (gamma) 
        for i, j in product(networks, networks):
            if networks.index(i) < networks.index(j):
                if k_idx == counter + 36:
                    feature_names.append(f'{i} & {j} in gamma')
                counter += 1
    
    if 72 <= k_idx < 87: # corresponds to aud network (alpha)
        for i, j in product(aud_network, aud_network):
            if aud_network.index(i) < aud_network.index(j):
                if k_idx == counter + 72:
                    feature_names.append(f'{i} & {j} in alpha')
                counter += 1
    
    if 87 <= k_idx < 102: # corresponds to aud network (gamma)
        for i, j in product(aud_network, aud_network):
            if aud_network.index(i) < aud_network.index(j):
                if k_idx == counter + 87:
                    feature_names.append(f'{i} & {j} in gamma')
                counter += 1


In [ ]:
len(feature_names)

In [ ]:
shap.summary_plot(shap_values[1], features=X_test, feature_names=feature_names,
                    max_display=20, alpha=0.1, cmap='coolwarm') # barplots

In [ ]:
np.save('/Users/payamsadeghishabestari/Prime/shap/network+aun/X_test.npy', np.array(X_test))

Plotting

In [ ]:
import nilearn
from nilearn.plotting import view_connectome, plot_connectome

In [ ]:
## plotting shap
X_test = np.load('/Users/payamsadeghishabestari/Prime/shap/network+aun/X_test.npy', allow_pickle=True)
shap_values = np.load('/Users/payamsadeghishabestari/Prime/shap/network+aun/shap.npy', allow_pickle=True)
feature_names = np.load('/Users/payamsadeghishabestari/Prime/shap/network+aun/features_name.npy', allow_pickle=True)

shap.summary_plot(shap_values[1], features=X_test, feature_names=feature_names,
                    max_display=15, alpha=0.1, cmap='coolwarm', plot_size=(11,15)) # barplots
plt.yticks(fontsize=9) 
plt.show()


In [ ]:
### extracting adjacency matrix in alpha and gamma
aud_network = ['superiorparietal-lh', 'superiorparietal-rh',
                'superiortemporal-lh', 'superiortemporal-rh',
                'transversetemporal-lh', 'transversetemporal-rh']
folder_path = '/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/within_aud_network'  
alpha_files = []; gamma_files = []
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    files_in_folder = sorted(os.listdir(folder_path), key=lambda x: os.path.getmtime(os.path.join(folder_path, x)))
    for filename in files_in_folder:
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and file_path.endswith(".npy"):
            if 'alpha' in file_path:
                alpha_files.append(file_path)
            if 'gamma' in file_path:
                gamma_files.append(file_path)

alpha_as = []
gamma_as = []
for file_a, file_g in zip(alpha_files, gamma_files):
    alpha_ss = np.load(file_a, allow_pickle=True)
    alpha_as.append(alpha_ss.mean(axis=0))
    gamma_ss = np.load(file_g, allow_pickle=True)
    gamma_as.append(gamma_ss.mean(axis=0))

alpha_con_aud = np.array(alpha_as).mean(axis=0)
gamma_con_aud = np.array(gamma_as).mean(axis=0)

### converting to mni coordinate
atlas = 'aparc'
labels_atlas = mne.read_labels_from_annot(subject='fsaverage', parc=atlas,
                                    subjects_dir=None, verbose=False)[:-1]
aud_lbs = []
for i in aud_network:
    for lb in labels_atlas:
        if i == lb.name:
            aud_lbs.append(lb)

node_coords = []
for label in aud_lbs:
    if label.hemi == 'lh':
        hemi = 0
    if label.hemi == 'rh':
        hemi = 1
    center_vertex = label.center_of_mass(subject='fsaverage', 
                                        restrict_vertices=False, 
                                        subjects_dir=None)
    mni_pos = mne.vertex_to_mni(center_vertex, hemis=hemi,
                                subject='fsaverage', subjects_dir=None)
    node_coords.append(mni_pos)
node_coords = np.array(node_coords)

### visualizing
adjacency_matrix1 = alpha_con_aud + alpha_con_aud.T
for i, j in product(range(6), range(6)):
    if (i, j)==(3, 5) or (i, j)==(1, 2):
        print('hello')
    else:
        adjacency_matrix1[i][j] = 0

adjacency_matrix1 = adjacency_matrix1 + adjacency_matrix1.T

adjacency_matrix2 = alpha_con_aud + alpha_con_aud.T
for i, j in product(range(6), range(6)):
    if (i, j)==(0, 3):
        print('hello')
    else:
        adjacency_matrix2[i][j] = 0

adjacency_matrix2 = adjacency_matrix2 + adjacency_matrix2.T

fig, axes = plt.subplots(1, 1, figsize=(10, 6))
plot_connectome(adjacency_matrix=adjacency_matrix1,
                node_coords=node_coords, node_color='k',
                edge_cmap='PuOr', node_size=30, axes=axes,
                alpha=0.7, edge_kwargs={'linewidth': 3},
                edge_threshold='0%', display_mode="lzry", edge_vmax=4,
                edge_vmin=-200)

plot_connectome(adjacency_matrix=adjacency_matrix2,
                node_coords=node_coords, node_color='k',
                edge_cmap='PuOr', node_size=30, axes=axes, 
                alpha=0.7, edge_kwargs={'linewidth': 3},
                edge_threshold='0%', display_mode="lzry", edge_vmax=15,
                edge_vmin=0)

In [ ]:
## plotting networks parcels
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc')[:-1] # 69 labels
bl_names = [bl.name for bl in brain_labels]
clr = 0.85 
silhouette_dict = {'color':(0.8, 0.8, 0.8), 'linewidth':1, 'alpha':0.8, 'decimate':0.9}
hemi = 'both'
color = 'red'
mode = 'rgb'
alpha = 0.6

# vsn
views = 'medial'
Brain = mne.viz.get_brain_class()
brain_kwargs = dict(alpha=1, background="white", cortex=[(clr, clr, clr), (clr, clr, clr)], size=(800, 600), views=views)
brain = Brain("fsaverage", hemi=hemi, surf="pial_semi_inflated", silhouette=silhouette_dict, **brain_kwargs)
for lb in vsn_lbs:
    for lb_n in bl_names:
        if lb == lb_n:
            idx = bl_names.index(lb)
            brain.add_label(label=brain_labels[idx], hemi=hemi, color=color, borders=False, alpha=alpha)
            brain.save_image(filename='/Users/payamsadeghishabestari/Prime/shap/network+aun/vsn.png', mode=mode)

# smn
views = 'dorsal'
Brain = mne.viz.get_brain_class()
brain_kwargs = dict(alpha=1, background="white", cortex=[(clr, clr, clr), (clr, clr, clr)], size=(800, 600), views=views)
brain = Brain("fsaverage", hemi=hemi, surf="pial_semi_inflated", silhouette=silhouette_dict, **brain_kwargs)
for lb in smn_lbs:
    for lb_n in bl_names:
        if lb == lb_n:
            idx = bl_names.index(lb)
            brain.add_label(label=brain_labels[idx], hemi=hemi, color=color, borders=False, alpha=alpha)
            brain.save_image(filename='/Users/payamsadeghishabestari/Prime/shap/network+aun/smn.png', mode=mode)


# aun
views = 'lateral'
Brain = mne.viz.get_brain_class()
brain_kwargs = dict(alpha=1, background="white", cortex=[(clr, clr, clr), (clr, clr, clr)], size=(800, 600), views=views)
brain = Brain("fsaverage", hemi=hemi, surf="pial_semi_inflated", silhouette=silhouette_dict, **brain_kwargs)
for lb in aun_lbs:
    for lb_n in bl_names:
        if lb == lb_n:
            idx = bl_names.index(lb)
            brain.add_label(label=brain_labels[idx], hemi=hemi, color=color, borders=False, alpha=alpha)
            brain.save_image(filename='/Users/payamsadeghishabestari/Prime/shap/network+aun/aun.png', mode=mode)

# van
views = 'lateral'
Brain = mne.viz.get_brain_class()
brain_kwargs = dict(alpha=1, background="white", cortex=[(clr, clr, clr), (clr, clr, clr)], size=(800, 600), views=views)
brain = Brain("fsaverage", hemi=hemi, surf="pial_semi_inflated", silhouette=silhouette_dict, **brain_kwargs)
for lb in van_lbs:
    for lb_n in bl_names:
        if lb == lb_n:
            idx = bl_names.index(lb)
            brain.add_label(label=brain_labels[idx], hemi=hemi, color=color, borders=False, alpha=alpha)
            brain.save_image(filename='/Users/payamsadeghishabestari/Prime/shap/network+aun/van.png', mode=mode)

# # dan
views = 'lateral'
Brain = mne.viz.get_brain_class()
brain_kwargs = dict(alpha=1, background="white", cortex=[(clr, clr, clr), (clr, clr, clr)], size=(800, 600), views=views)
brain = Brain("fsaverage", hemi=hemi, surf="pial_semi_inflated", silhouette=silhouette_dict, **brain_kwargs)
for lb in dan_lbs:
    for lb_n in bl_names:
        if lb == lb_n:
            idx = bl_names.index(lb)
            brain.add_label(label=brain_labels[idx], hemi=hemi, color=color, borders=False, alpha=alpha)
            brain.save_image(filename='/Users/payamsadeghishabestari/Prime/shap/network+aun/dan.png', mode=mode)

# # fpn
views = 'dorsal'
Brain = mne.viz.get_brain_class()
brain_kwargs = dict(alpha=1, background="white", cortex=[(clr, clr, clr), (clr, clr, clr)], size=(800, 600), views=views)
brain = Brain("fsaverage", hemi=hemi, surf="pial_semi_inflated", silhouette=silhouette_dict, **brain_kwargs)
for lb in fpn_lbs:
    for lb_n in bl_names:
        if lb == lb_n:
            idx = bl_names.index(lb)
            brain.add_label(label=brain_labels[idx], hemi=hemi, color=color, borders=False, alpha=alpha)
            brain.save_image(filename='/Users/payamsadeghishabestari/Prime/shap/network+aun/fpn.png', mode=mode)

# dmn
views = 'medial'
Brain = mne.viz.get_brain_class()
brain_kwargs = dict(alpha=1, background="white", cortex=[(clr, clr, clr), (clr, clr, clr)], size=(800, 600), views=views)
brain = Brain("fsaverage", hemi='lh', surf="pial_semi_inflated", silhouette=silhouette_dict, **brain_kwargs)
for lb in dmn_lbs:
    for lb_n in bl_names:
        if lb == lb_n and lb[-2:]=='lh':
            idx = bl_names.index(lb)
            brain.add_label(label=brain_labels[idx], hemi='hemi', color=color, borders=False, alpha=alpha)
            brain.save_image(filename='/Users/payamsadeghishabestari/Prime/shap/network+aun/dmn.png', mode=mode)

# dgn
views = 'medial'
Brain = mne.viz.get_brain_class()
brain_kwargs = dict(alpha=1, background="white", cortex=[(clr, clr, clr), (clr, clr, clr)], size=(800, 600), views=views)
brain = Brain("fsaverage", hemi='lh', surf="pial_semi_inflated", silhouette=silhouette_dict, **brain_kwargs)
for lb in dgn_lbs:
    for lb_n in bl_names:
        if lb == lb_n and lb[-2:]=='lh':
            idx = bl_names.index(lb)
            brain.add_label(label=brain_labels[idx], hemi='hemi', color=color, borders=False, alpha=alpha)
            brain.save_image(filename='/Users/payamsadeghishabestari/Prime/shap/network+aun/dgn.png', mode=mode)

# lbn
views = 'medial'
Brain = mne.viz.get_brain_class()
brain_kwargs = dict(alpha=1, background="white", cortex=[(clr, clr, clr), (clr, clr, clr)], size=(800, 600), views=views)
brain = Brain("fsaverage", hemi='lh', surf="pial_semi_inflated", silhouette=silhouette_dict, **brain_kwargs)
for lb in lbn_lbs:
    for lb_n in bl_names:
        if lb == lb_n and lb[-2:]=='lh':
            idx = bl_names.index(lb)
            brain.add_label(label=brain_labels[idx], hemi='hemi', color=color, borders=False, alpha=alpha)
            brain.save_image(filename='/Users/payamsadeghishabestari/Prime/shap/network+aun/lbn.png', mode=mode)

In [ ]:
### extracting adjacency matrix in alpha and gamma for networks
folder_path = '/Users/payamsadeghishabestari/eeg_data/Regensburg/connectivity/networks_connectivity'  
alpha_files = []; gamma_files = []
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    files_in_folder = sorted(os.listdir(folder_path), key=lambda x: os.path.getmtime(os.path.join(folder_path, x)))
    for filename in files_in_folder:
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and file_path.endswith(".npy"):
            if 'alpha' in file_path:
                alpha_files.append(file_path)
            if 'gamma' in file_path:
                gamma_files.append(file_path)

alpha_as = []
gamma_as = []
for file_a, file_g in zip(alpha_files, gamma_files):
    alpha_ss = np.load(file_a, allow_pickle=True)
    alpha_as.append(alpha_ss.mean(axis=0))
    gamma_ss = np.load(file_g, allow_pickle=True)
    gamma_as.append(gamma_ss.mean(axis=0))

alpha_con_net = np.array(alpha_as).mean(axis=0)
gamma_con_net = np.array(gamma_as).mean(axis=0)

## plotting network circular graph
# color='#9467bd' # #ff7f0e
networks = ['vsn', 'smn', 'aun', 'van', 'dan', 'fpn', 'dmn', 'dgn', 'lbn']
set2_palette = sns.color_palette("tab10")

for i, j in product(range(9), range(9)):
    if not ((i, j)==(6, 3) or (i, j)==(4, 2) or (i, j)==(7, 3)):
        alpha_con_net[i][j] = 0

for i, j in product(range(9), range(9)):
    if not ((i, j)==(2, 1) or (i, j)==(7, 3) or (i, j)==(8, 2) or (i, j)==(5, 2) or (i, j)==(7, 2)):
        gamma_con_net[i][j] = 0

gamma_con_net = np.zeros(shape=(9, 9))

gamma_con_net[2][1] = 1
gamma_con_net[7][3] = -1
gamma_con_net[8][2] = 1
gamma_con_net[5][2] = -1
gamma_con_net[7][2] = 1
gamma_con_net[5][1] = 1
gamma_con_net[4][3] = 1
gamma_con_net[8][3] = -1
gamma_con_net[3][1] = -1


fig, ax = plt.subplots(1,1, figsize=(11,11), subplot_kw={'projection': 'polar'})
plot_connectivity_circle(con=alpha_con_net, node_names=networks, n_lines=3, vmin=-1, vmax=100,
                        textcolor='black', node_edgecolor='white', colormap='coolwarm',
                        fontsize_names=10, title=None, facecolor='white',
                        colorbar=False, linewidth=2.5, node_width=30,
                        node_height=1.8, node_colors=set2_palette, ax=ax)

# plot_connectivity_circle(con=gamma_con_net, node_names=networks, n_lines=5, vmin=-10, vmax=0,
#                         textcolor='black', node_edgecolor='white', colormap='PuOr',
#                         fontsize_names=10, title=None, facecolor='white',
#                         colorbar=False, linewidth=2.5, node_width=30,
#                         node_height=1.8, node_colors=set2_palette, ax=ax)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
shap.dependence_plot("Alpha power", list(shap_new)[1], X_test_new, alpha=0.7,
                    feature_names=feature_names, interaction_index="Gamma power",
                    cmap='coolwarm', ax=ax)
ax.set_ylim([-0.3, 0.3])
ax.hlines(y=0, xmin=0 , xmax=1.1, colors='k', linestyles='--')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
shap.dependence_plot("Gamma power", list(shap_new)[1], X_test_new, alpha=0.7,
                    feature_names=feature_names, interaction_index="Gamma entropy",
                    cmap='coolwarm', ax=ax)
ax.set_ylim([-0.3, 0.3])
ax.hlines(y=0, xmin=0 , xmax=1.1, colors='k', linestyles='--')

In [ ]:
shap.summary_plot(list(shap_new)[1], features=X_test_new, feature_names=feature_names, alpha=0.1,
                    max_display=12, cmap='coolwarm', plot_size=(8,5)) # barplots

SHAP plot for Source space

In [ ]:
# random forest
n_sel_features = 100
model = RandomForestClassifier(n_estimators=100)
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=None)
mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1

n_scores = cross_val_score(model, X_train, y_train, groups = group_train ,scoring='accuracy', cv=10, n_jobs=-1)
model.fit(X_train, y_train)

# reporting and features overview
importance = model.feature_importances_
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')

selected_features_idx = np.argsort(importance)[-n_sel_features:]

# results align with the plot
brain_labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc', verbose=False)
for index, imp in zip(np.array(keep_idxs)[selected_features_idx], np.sort(importance)[-n_sel_features:][:]):
    if 2*68 <= index < 3*68:
        clas = 'alpha'
        new_index = index - 2*68
        print(f"{brain_labels[new_index].name} in class {clas} with importance: {round(imp, 3)}")
    
    if 4*68 <= index < 5*68:
        clas = 'gamma'  
        new_index = index - 4*68 
        print(f"{brain_labels[new_index].name} in class {clas} with importance: {round(imp, 3)}")

# only select some indexes and plot summaryplot for them

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer.shap_values(X_test) # computed for all features

In [ ]:
shap_values[0].shape

In [ ]:
x = shap.summary_plot(shap_values[1], features=X_test, feature_names=None,
                    max_display=20, alpha=0.1, cmap='coolwarm') # barplots

Shuffling the Clf results and recheck

In [ ]:
## original classification
n_sel_features = 100
model = RandomForestClassifier(n_estimators=100)
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=None)
mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1

n_scores = cross_val_score(model, X_train, y_train, groups = group_train ,scoring='accuracy', cv=10, n_jobs=-1)
model.fit(X_train, y_train)

# reporting and features overview
importance = model.feature_importances_
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')

## after shuffling
label_array_copy = label_array
random.shuffle(label_array_copy)

n_sel_features = 100
model = RandomForestClassifier(n_estimators=100)
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(features_array_after_corr,
                                                                            label_array_copy, group_array,
                                                                            test_size=0.3, train_size=0.7,
                                                                            shuffle = True, random_state=None)
mask_train = np.isnan(X_train)
mask_test = np.isnan(X_test)
nan_indices_train = np.where(mask_train)
nan_indices_test = np.where(mask_test)
for i in range(len(nan_indices_train[0])):
    X_train[nan_indices_train[0][i]][nan_indices_train[1][i]] = 1
for i in range(len(nan_indices_test[0])):
    X_test[nan_indices_test[0][i]][nan_indices_test[1][i]] = 1

n_scores = cross_val_score(model, X_train, y_train, groups = group_train ,scoring='accuracy', cv=10, n_jobs=-1)
model.fit(X_train, y_train)

# reporting and features overview
importance = model.feature_importances_
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy of the model checked with test data without eliminating features: {accuracy}')

Stats on Variables

In [ ]:
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency

In [ ]:
df_aaeeg = pd.read_excel('/Users/payamsadeghishabestari/eeg_data/Regensburg/transfer_147155_files_ac07b771/aaeeg_rsEEG/AAEEG_TL_data.xlsx')
df_neuropren = pd.read_excel('/Users/payamsadeghishabestari/eeg_data/Regensburg/transfer_147155_files_ac07b771/neuropren_rsEEG/neuropren_TL_data.xlsx')
df = pd.read_excel('/Users/payamsadeghishabestari/Prime/PRIME_beh-data_complete.xlsx')

df_aaeeg['WN_TL_0'] = df_aaeeg['WN_TL_0'].apply(lambda x: 0 if x > 90 else 1)
df_neuropren['b1_WN_0'] = df_neuropren['b1_WN_0'].apply(lambda x: 0 if x > 90 else 1)
df_aaeeg = df_aaeeg.rename(columns={'WN_TL_0': 'BATS'})
df_neuropren = df_neuropren.rename(columns={'b1_WN_0': 'BATS'})
df1 = df_aaeeg[["random_id", "BATS"]]
df2 = df_neuropren[["random_id", "BATS"]]
df3 = pd.concat((df1, df2))
df = df.merge(right=df3, on="random_id")
df1 = df[df["BATS"] == 1]
df2 = df[df["BATS"] == 0]

In [ ]:
df2["sex"].value_counts()

In [ ]:
for column in df1.columns[5:-1]:
    print(f"{column} -> mean: {round(df1[column].mean(), 2)}, std: {round(df1[column].std(), 2)}")

In [ ]:
for column in df1.columns[5:-1]:
    print(f"{column} -> mean: {round(df2[column].mean(), 2)}, std: {round(df2[column].std(), 2)}")

In [ ]:
t_p_values = {}
for column in df1.columns[5:-1]:
    if column == "sex":
        contingency_table = pd.crosstab(df[column], df["BATS"])
        t_statistic, p_value, _, _ = chi2_contingency(contingency_table)
    else:
        t_statistic, p_value = ttest_ind(df1[column], df2[column], permutations=0)
    
    t_p_values[column] = {'t-statistic': round(t_statistic, 3), 'p-value': round(p_value, 3)}

# Convert the dictionary to a DataFrame
result_df = pd.DataFrame(t_p_values)